In [4]:
import pandas as pd
import numpy as np
import pickle
import re

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
from IPython.display import display
from sklearn.pipeline import Pipeline

from datetime import datetime

In [5]:
# Tüm sütunları görmek için sütun sayısı sınırlamasını kaldır
pd.set_option('display.max_columns', None)
pd.options.display.max_columns = None

In [ ]:
df = pd.read_csv('sample.csv', index_col='Unnamed: 0')
df.shape

<ipython-input-62-aea0ff63ee93>:1: DtypeWarning: Columns (109) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('sample.csv', index_col='Unnamed: 0')


(60213, 202)

In [ ]:
col_list = ['campaign_1',
'campaign_2',
'campaign_3',
'campaign_4',
'campaign_5',
'campaign_6',
'campaign_7',
'campaign_8',
'campaign_9',
'campaign_10',
'campaign_11',
'campaign_12',
'campaign_13',
'campaign_14',
'campaign_15',
'campaign_16']

# Rows which has at least one campaign interaction
rows_with_more_than_one_1 = df[(df[col_list].sum(axis=1) >= 1)]

print(len(rows_with_more_than_one_1))

9241


In [7]:
"""
This function is designed to reduce memory usage by implementing memory-efficient techniques.
You can provide additional details and explanations here.
"""
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
df = reduce_mem_usage(df)

Mem. usage decreased to 29.98 Mb (67.9% reduction)


In [ ]:
# Check for null values in the specified column
null_count = df.iloc[:, :50].isnull().sum()

# Display the result
print(f"The number of null values column is: {null_count}")

The number of null values column is: SubscriberID            0
SnapshotDate            0
campaign_1              0
campaign_2              0
campaign_3              0
campaign_4              0
campaign_5              0
campaign_6              0
campaign_7              0
campaign_8              0
campaign_9              0
campaign_10             0
campaign_11             0
campaign_12             0
campaign_13             0
campaign_14             0
campaign_15             0
campaign_16             0
NumVoicePrepaid       690
NumVoicePostpaid      690
VoiceOnNet            690
VoiceOffNet           690
VoiceInt              690
NumCalls              690
CallDuration          690
NumSMSPrepaid       30209
NumSMSPostpaid      30209
SMSOnNet            30209
SMSOffNet           30209
SMSInt              30209
NumSMS              30209
SMSLength           30209
NumMMSPrepaid       58313
NumMMSPostpaid      58313
MMSOnNet            58313
MMSOffNet           58313
MMSInt              58313
N

In [ ]:
column_list = df.columns.tolist()
campaign_list = ['campaign_1',
'campaign_2',
'campaign_3',
'campaign_4',
'campaign_5',
'campaign_6',
'campaign_7',
'campaign_8',
'campaign_9',
'campaign_10',
'campaign_11',
'campaign_12',
'campaign_13',
'campaign_14',
'campaign_15',
'campaign_16']

In [ ]:
# Get items from the list that end with 'Date'
date_list = [item for item in column_list if item.endswith('Date')]

# Display the result
print(date_list)

['SnapshotDate', 'StartDate', 'EndDate', 'PackagePlanSubscriptionDate', 'BirthDate', 'DaysToActiveStopDate', 'DaysToSuspendStopDate', 'DaysToDisableStopDate']


In [ ]:
# Add 'RechargeMonth' to the datelist
date_list.append('RechargeMonth')

In [ ]:
# Calculate the day difference between the current date and the provided target date.

def calculate_day_difference(df, column_name):
    print('Day diff started')
    today = datetime.today().date()
    #df[column_name] = pd.to_datetime(df[column_name])

    try:
        df[column_name] = pd.to_datetime(df[column_name]).dt.date
        #df[column_name] = pd.to_datetime(df[column_name])
        df[column_name] = (today - df[column_name]).dt.days

    except (TypeError, ValueError):
        print(f"Error: Unable to convert column '{column_name}' to datetime.")

    return df

In [ ]:
for date in date_list:
  print(date)
  calculate_day_difference(df, date)

SnapshotDate
Day diff started
StartDate
Day diff started
EndDate
Day diff started
PackagePlanSubscriptionDate
Day diff started
BirthDate
Day diff started
DaysToActiveStopDate
Day diff started
DaysToSuspendStopDate
Day diff started
DaysToDisableStopDate
Day diff started
RechargeMonth
Day diff started


In [ ]:
def get_object_columns(df):
    """
    Get object-type columns from a DataFrame.

    Parameters:
    - df: pandas DataFrame

    Returns:
    - list of column names with object dtype
    """
    return df.select_dtypes(include='object').columns.tolist()

In [ ]:
# Get object-type columns from the DataFrame
object_columns = get_object_columns(df)

# Display the result
print("Object-type columns:", object_columns)

Object-type columns: ['Cycle', 'CustomerStatus', 'CustomerTypeGroup', 'CustomerSegment', 'SalesChannel', 'Gender', 'MaritalStatus', 'Nationality', 'Occupation']


In [ ]:
def extract_numeric_values(value):
    """
    Extract numeric values from a string.

    Parameters:
    - value: input string with numeric value and unit

    Returns:
    - int or float: extracted numeric value
    """
    # Use regular expression to find numeric part
    match = re.search(r'\b(\d+(\.\d+)?)\b', value)

    if match:
        # Convert the matched group to float
        return float(match.group(1))
    else:
        # If no match, return None or raise an exception based on your preference
        return None

In [ ]:
# Apply the function to the 'Duration' column
df['Cycle'] = df['Cycle'].astype('str')
df['Cycle'] = df['Cycle'].apply(extract_numeric_values)

# Display the result
df[object_columns].head(10)

,Cycle,CustomerStatus,CustomerTypeGroup,CustomerSegment,SalesChannel,Gender,MaritalStatus,Nationality,Occupation
1,NaN,Active,Individual,Individual,SalesChannel_Unknown,Male,NaN,Indian,Accountant
2,NaN,Active,Individual,Individual,SalesChannel_Unknown,NaN,NaN,German,Student
3,NaN,Active,Individual,Individual,SalesChannel_Unknown,NaN,NaN,German,Labour worker
4,NaN,Active,Individual,Individual,SalesChannel_Unknown,NaN,NaN,German,Others
5,NaN,Active,Individual,Individual,SalesChannel_Unknown,NaN,NaN,German,Others
6,NaN,Active,Individual,Individual,SalesChannel_Unknown,NaN,NaN,German,Military
7,NaN,Active,Individual,Individual,SalesChannel_Unknown,NaN,NaN,German,Others
8,NaN,Active,Individual,Individual,SalesChannel_Unknown,NaN,NaN,German,Teacher
9,NaN,Active,Individual,Individual,SalesChannel_Unknown,NaN,NaN,German,Labour worker
10,NaN,Active,Individual,Individual,SalesChannel_Unknown,NaN,NaN,German,Others


In [ ]:
get_dummy_cols = ['MaritalStatus', 'SalesChannel', 'Occupation',
                  'Gender', 'CustomerTypeGroup', 'CustomerSegment', 'CustomerStatus']

In [ ]:
# Get dummy variables for the 'Category' columns
for col in get_dummy_cols:
    df = pd.concat([df, pd.get_dummies(df[col], prefix=col + "_cat")], axis=1)
    df.drop(columns=[col], axis=1, inplace=True)
print("Get Dummies pass")

Get Dummies pass


In [ ]:
df = reduce_mem_usage(df)

Mem. usage decreased to 28.31 Mb (10.8% reduction)


In [ ]:
def less_than_10_occurrences(df, column_name):
    """
    Return a list of values that occur less than 10 times in the specified column.

    Parameters:
    - df: pandas DataFrame
    - column_name: column name for which to check occurrences

    Returns:
    - list: values occurring less than 10 times
    """
    # Count occurrences of each unique value in the specified column
    value_counts = df[column_name].value_counts()

    # Filter values that occur less than 10 times
    less_than_10_occurrences = value_counts[value_counts < 10].index.tolist()

    # Replace values with 'other' if they occur less than 10 times
    df[column_name] = df[column_name].apply(lambda x: 'other' if x in less_than_10_occurrences else x)


    return less_than_10_occurrences

In [ ]:
# Get values occurring less than 10 times in the 'YourColumn' column
rare_nationalities = less_than_10_occurrences(df, 'Nationality')

# Display the result
print("Values occurring less than 10 times:", rare_nationalities)

Values occurring less than 10 times: ['Other', 'Spanish', 'Polish', 'Belarusian', 'Armenian', 'Uzbekistani', 'Ugandan', 'Belgian', 'Bahamian', 'New Zealander', 'Qatari', 'Cameroonian', 'Macedonian', 'Swedish', 'Venezuelan', 'Emirian', 'Mexican', 'Seychellois', 'Japanese', 'Zimbabwean', 'Azerbaijani', 'Tanzanian', 'Danish', 'Cypriot', 'Romanian', 'Albanian', 'Serbian', 'Angolan', 'Czech', 'Somali', 'Slovakian Republic', 'Nigerian', 'Colombian', 'Bosnian', 'Austrian', 'U.S.Virgin Islands', 'South Korean', 'Croatian', 'Fijian', 'Chilean', 'Monacan', 'Sierra Leonean', 'Hungarian', 'Taiwanese', 'UAE (United Arab Emirates)', 'Singaporean', 'Palauan', 'Maltese', 'Kyrgyz', 'Eritrean', 'Belizean', 'Peruvian', 'Falkland Islands', 'Kazakhstani', 'Mauritian', 'Argentinean', 'Slovakian', 'Uruguayan', 'Ecuadorean', 'Maldivan', 'Tunisian', 'Afghani', 'Solomon Islander', 'Mauritanian', 'Hong Kong', 'Argentinian', 'Republic Of Montenegro', 'Guatemalan', 'Guinea-Bissauan', 'Burkinabe', 'Somalian', 'Ando

In [ ]:
def scale_and_encode(df, column_name):
  # Calculate the normalized frequencies of the 'column_name' column
  normalized_counts = df[column_name].value_counts(normalize=True).sort_index(ascending=False)

  # Create a DataFrame containing the normalized values and frequencies
  result_df = pd.DataFrame({
      'Normalized_Value': normalized_counts.index,
      'Frequency': normalized_counts.values
  })

  # Replace the 'column_name' column with normalized frequencies
  df[column_name] = df[column_name].map(result_df.set_index('Normalized_Value')['Frequency'])

  # Scale the 'column_name' column between 0 and 1 using MinMaxScaler
  scaler = MinMaxScaler()
  df[column_name] = scaler.fit_transform(df[[column_name]])

  return df

In [ ]:
# Scale and encode the 'Nationality' column
df = scale_and_encode(df, 'Nationality')

In [ ]:
# Columns which has bigger than 1
columns_with_max_greater_than_1 = df.columns[df.max() > 1].tolist()

# Display columns
print("Column lists :", columns_with_max_greater_than_1)

Column lists : ['SubscriberID', 'SnapshotDate', 'NumVoicePrepaid', 'NumVoicePostpaid', 'VoiceOnNet', 'VoiceOffNet', 'VoiceInt', 'NumCalls', 'CallDuration', 'NumSMSPrepaid', 'NumSMSPostpaid', 'SMSOnNet', 'SMSOffNet', 'SMSInt', 'NumSMS', 'SMSLength', 'NumMMSPrepaid', 'NumMMSPostpaid', 'MMSOnNet', 'MMSOffNet', 'MMSInt', 'NumMMS', 'MMSLength', 'NumDataSPrepaid', 'NumDataPostpaid', 'DataCharTraf', 'NumDataSessions', 'SessionsLength', 'TotalFlux', 'NumMonPrepaid', 'NumMonPostpaid', 'OrderNorm', 'OrderExp', 'OrderDay', 'NumOrders', 'NumMGRPrepaid', 'TransferAmount', 'ChargeAmount', 'ChargeFromPrepaid', 'PrepaidBalance', 'NumVouPrepaid', 'NumRechargeMine', 'NumRecharges', 'RechargeValue', 'PrepaidBalanceBefore', 'RechargeForPrepaid', 'StartDate', 'EndDate', 'ContractLengthD', 'ContractLengthM', 'CycleDetails', 'Cycle', 'PackagePlanSubscriptionDate', 'BirthDate', 'ActualMoUM_PRev', 'CieledMoUM_PRev', 'ChargeDurationM_PRev', 'DownFlux_PRev', 'ActualMoUM_PoRev', 'CieledMoUM_PoRev', 'ChargeDuratio

In [ ]:
#Drop SubscriberID from list

item_to_remove = 'SubscriberID'

my_list = [item for item in columns_with_max_greater_than_1 if item != item_to_remove]

print(my_list)

['SnapshotDate', 'NumVoicePrepaid', 'NumVoicePostpaid', 'VoiceOnNet', 'VoiceOffNet', 'VoiceInt', 'NumCalls', 'CallDuration', 'NumSMSPrepaid', 'NumSMSPostpaid', 'SMSOnNet', 'SMSOffNet', 'SMSInt', 'NumSMS', 'SMSLength', 'NumMMSPrepaid', 'NumMMSPostpaid', 'MMSOnNet', 'MMSOffNet', 'MMSInt', 'NumMMS', 'MMSLength', 'NumDataSPrepaid', 'NumDataPostpaid', 'DataCharTraf', 'NumDataSessions', 'SessionsLength', 'TotalFlux', 'NumMonPrepaid', 'NumMonPostpaid', 'OrderNorm', 'OrderExp', 'OrderDay', 'NumOrders', 'NumMGRPrepaid', 'TransferAmount', 'ChargeAmount', 'ChargeFromPrepaid', 'PrepaidBalance', 'NumVouPrepaid', 'NumRechargeMine', 'NumRecharges', 'RechargeValue', 'PrepaidBalanceBefore', 'RechargeForPrepaid', 'StartDate', 'EndDate', 'ContractLengthD', 'ContractLengthM', 'CycleDetails', 'Cycle', 'PackagePlanSubscriptionDate', 'BirthDate', 'ActualMoUM_PRev', 'CieledMoUM_PRev', 'ChargeDurationM_PRev', 'DownFlux_PRev', 'ActualMoUM_PoRev', 'CieledMoUM_PoRev', 'ChargeDurationM_PoRev', 'DownFlux_PoRev', 'B

In [ ]:
# MinMaxScaler for nornalization numeric columns between 0 and 1

scaler = MinMaxScaler()

for col in my_list:
  df[col] = scaler.fit_transform(df[[col]])

In [ ]:
# Method for filling the null values
def fill_null_values(df):
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column].fillna('unknown', inplace=True)
        elif df[column].dtype in ['int8', 'int16', 'int32', 'int64', 'float64', 'float32', 'float16', 'uint8']:
            df[column].fillna(0, inplace=True)
            df[column][np.isnan(df[column])] = 0
    return df

In [ ]:
df = fill_null_values(df)


<ipython-input-86-760b3b481739>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column][np.isnan(df[column])] = 0
<ipython-input-86-760b3b481739>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column][np.isnan(df[column])] = 0
<ipython-input-86-760b3b481739>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column][np.isnan(df[column])] = 0
<ipython-input-86-760b3b481739>:8: SettingWithCopyWarning: 
A value is trying to be s

In [ ]:
print(campaign_list)

['campaign_1', 'campaign_2', 'campaign_3', 'campaign_4', 'campaign_5', 'campaign_6', 'campaign_7', 'campaign_8', 'campaign_9', 'campaign_10', 'campaign_11', 'campaign_12', 'campaign_13', 'campaign_14', 'campaign_15', 'campaign_16']


In [ ]:
df.shape

(60213, 264)

In [ ]:
X = df.drop(columns=campaign_list) #features (independent variables)
X.shape

(60213, 248)

In [ ]:
# Her eleman için yeni DataFrame oluşturma ve kaydetme
for campaign_name in campaign_list:
    col_list = list(set(df.columns) - set(campaign_list))
    new_df = df[col_list].copy()
    #new_df = df[list(set(df.columns) - set(campaign_list))].copy()
    new_df[[campaign_name]] = df[[campaign_name]].copy()
    new_df.columns = list(set(df.columns) - set(campaign_list)) + [campaign_name]  # Yeni kolon adı vermek istiyorsanız değiştirebilirsiniz
    globals()[f"df_{campaign_name}"] = new_df  # DataFrame'i global olarak adlandırma


# For example
#print(df_campaign_1)
#print(df_campaign_2)
# ...
#print(df_campaign_16)

In [ ]:
df_campaign_1.head()

,allpresure2,Occupation_cat_D,ChurnPresure1,NumVouHybrid,PrepaidRevenue,NumberOfUpgrades,AddOnSubscriptionRevenue,NumMonPrepaid,CustomerSegment_cat_C,VouchersCount,MMSOnNet,OrderBill,Occupation_cat_Electrician,CustomerSegment_cat_Individual,Occupation_cat_Foreign Navy,DownFlux_PRev,bet,VASRevenue,Nationality,MaritalStatus_cat_Married,StartDate,deg_in,CustomerStatus_cat_Suspended,Occupation_cat_Accountant,NumberOfEnquiries,Occupation_cat_Architect,OtherRevenue,Occupation_cat_Specialist/Analyst/Advisor,HTM70CreditLimitBreach,SalesChannel_cat_Alternative,BilledMonth,TotalAmount_PostP,AmountOfCreditTransferTransactions,SalesChannel_cat_LMRA,CustomerSegment_cat_B,NumOrders,OrderWeek,CustomerSegment_cat_Medium SME,DaysToActiveStopDate,CDRBonusRevenueM_PoRev,Andpresure1,Occupation_cat_Police Officer,NumMMSPostpaid,Occupation_cat_Senior manager,RechargeForPrepaid,ETopUpCount,MigrationIN,NumSMS,NumberOfEnrollment,Occupation_cat_Military,PostpaidBalance,DataCharTraf,PostpaidDataRevenue,InterconnectPayableRevenue,eig_whole,RGE90Subscriberbase,CustomerSegment_cat_Large Enterprises,NumMMSPrepaid,ETopUp,TotalNETopUpCardCount,VoiceOnNet,CustomerTypeGroup_cat_Business Customer,ContractLengthD,CustomerSegment_cat_SME Flexible,PostpaidAddonDeActivations,OrderDay,iospresure2,NumVouPrepaid,DaysToSuspendStopDate,ChargeDurationM_PRev,TotalNETopUpCardAmount,eig_dir,ARPU,CustomerStatus_cat_Active,NumCalls,TotalMNPRequest,Occupation_cat_Dentist,LoyaltyPoints,Occupation_cat_A,Occupation_cat_Teacher,InterconnectReceivableRevenueRequest,NumberOfIncidents,CallDuration,allpresure1,OrderMonth,ChurnPresure2,CieledMoUM_PoRev,CustomerTypeGroup_cat_Individual,Occupation_cat_Nurse/Other medical staff,NumMGRPrepaid,Occupation_cat_Security/Guard,Occupation_cat_B,NumVouPostpaid,UpFlux_PoRev,NoOfEvents,NumSMSPrepaid,Occupation_cat_Sales Representative,UpFlux_PRev,Occupation_cat_Labour Worker,HTM100CreditLimitBreach,Occupation_cat_Labour worker,Gender_cat_Male,DownFlux_PoRev,DataRevenue,Occupation_cat_CEO,TotalNRechargesUpto2,BirthDate,ChargeFromPostpaid,SMSOnNet,ChargeFromPrepaid,VoiceRevenue,SalesChannel_cat_SalesChannel_Unknown,SMSRevenue,SubscriberID,Occupation_cat_Engineer,NumberOfTroubleTickets,PostpaidBalanceBefore,RechargeValue,GlobalVouchers,Occupation_cat_Mechanic,PackagePlanSubscriptionDate,CustomerStatus_cat_Inactive,NumberOfServiceRequests,TotalNRechargesUpto3,TotalAmount_PrePa,OrderExp,RGE30Subscriberbase,EoP_SubscribersApOG,NumberOfTicketsOpenedToday,CustomerSegment_cat_Contractor,VoiceOffNet,ActualMoUM_PoRev,SnapshotDate,MMSInt,TotalFlux,NumRechargeMine,CustomerSegment_cat_Government,MaritalStatus_cat_Single,NumVoicePrepaid,MMSLength,Occupation_cat_C,PrepaidBalance,NumberOfTermination,RoamingOutboundRevenue,Occupation_cat_Manager,SalesChannel_cat_UniConnect,OrderNULL,CustomerSegment_cat_Powered MEs,ChargeAmount,RoamingOutound,Occupation_cat_Senior Manager,EoPProvisionedBase,TotalNRechargesUpto250,CycleDetails,CustomerSegment_cat_Employees,MMSRevenue,postpaidAddonRevenue,NumDataSPrepaid,CustomerSegment_cat_D,NumRecharges,RechargeMonth,Occupation_cat_Banker,Occupation_cat_Lawyer,NumMonNULL,ContractLengthM,CustomerStatus_cat_New,SessionsLength,NoEventsVASPost,Occupation_cat_Consultant,PostpaidMMSRev,CustomerSegment_cat_Test,TotalNRechargesUpto5,PromiseToPay,CieledMoUM_PRev,Occupation_cat_Secretary,Andpresure2,CustomerSegment_cat_Special,Occupation_cat_Biologist/Chemist/Physician,ChargeDurationM_PoRev,Vouchers,deg_out,NumSMSPostpaid,TotalNRechargesUpto05,PrepaidBalanceBefore,LevelPoints,PostpaidVoiceRevenue,CustomerSegment_cat_Small SME,HTM90CreditLimitBreach,ExpiredPoints,Gender_cat_Female,VoiceInt,NumVoicePostpaid,IsCommitment,NumMMS,TotalNRechargesUpto25,Occupation_cat_Doctor,Occupation_cat_Supervisor,NumDataPostpaid,Occupation_cat_Pilot,EndDate,Occupation_cat_Student,PostpaidRevenues,ReadeemablePoints,MMSOffNet,Occupation_cat_Office Clerk/Office worker,Occupation_cat_Technician,eig_simple,SMSLength,NumberOfDowngrades,NumberOfCreditTransferTransactions

In [ ]:
# Define the target ratio (1-to-4)
target_ratio = 0.25

In [ ]:

# Separate majority and minority classes
majority_class = df_campaign_1[df_campaign_1['campaign_1'] == 0]
minority_class = df_campaign_1[df_campaign_1['campaign_1'] == 1]

# Calculate the number of minority samples needed for the target ratio
target_minority_count = int(len(majority_class) * target_ratio)

# Upsample minority class to reach the target ratio
minority_upsampled = resample(minority_class, replace=True, n_samples=target_minority_count, random_state=42)

# Combine the majority class with the upsampled minority class
camp1_oversampled_train_df = pd.concat([majority_class, minority_upsampled])

# Display the original and oversampled DataFrames
print("Original DataFrame:")
print(df_campaign_1.shape)

print("\nOversampled DataFrame:")
print(camp1_oversampled_train_df.shape)


Original DataFrame:
(60213, 249)

Oversampled DataFrame:
(74130, 249)


In [ ]:

# Separate majority and minority classes
majority_class = df_campaign_2[df_campaign_2['campaign_2'] == 0]
minority_class = df_campaign_2[df_campaign_2['campaign_2'] == 1]

# Calculate the number of minority samples needed for the target ratio
target_minority_count = int(len(majority_class) * target_ratio)

# Upsample minority class to reach the target ratio
minority_upsampled = resample(minority_class, replace=True, n_samples=target_minority_count, random_state=42)

# Combine the majority class with the upsampled minority class
camp2_oversampled_train_df = pd.concat([majority_class, minority_upsampled])

# Display the original and oversampled DataFrames
print("Original DataFrame:")
print(df_campaign_2.shape)

print("\nOversampled DataFrame:")
print(camp2_oversampled_train_df.shape)


Original DataFrame:
(60213, 249)

Oversampled DataFrame:
(72725, 249)


In [ ]:

# Separate majority and minority classes
majority_class = df_campaign_3[df_campaign_3['campaign_3'] == 0]
minority_class = df_campaign_3[df_campaign_3['campaign_3'] == 1]

# Calculate the number of minority samples needed for the target ratio
target_minority_count = int(len(majority_class) * target_ratio)

# Upsample minority class to reach the target ratio
minority_upsampled = resample(minority_class, replace=True, n_samples=target_minority_count, random_state=42)

# Combine the majority class with the upsampled minority class
camp3_oversampled_train_df = pd.concat([majority_class, minority_upsampled])

# Display the original and oversampled DataFrames
print("Original DataFrame:")
print(df_campaign_3.shape)

print("\nOversampled DataFrame:")
print(camp3_oversampled_train_df.shape)


Original DataFrame:
(60213, 249)

Oversampled DataFrame:
(71331, 249)


In [ ]:

# Separate majority and minority classes
majority_class = df_campaign_4[df_campaign_4['campaign_4'] == 0]
minority_class = df_campaign_4[df_campaign_4['campaign_4'] == 1]

# Calculate the number of minority samples needed for the target ratio
target_minority_count = int(len(majority_class) * target_ratio)

# Upsample minority class to reach the target ratio
minority_upsampled = resample(minority_class, replace=True, n_samples=target_minority_count, random_state=42)

# Combine the majority class with the upsampled minority class
camp4_oversampled_train_df = pd.concat([majority_class, minority_upsampled])

# Display the original and oversampled DataFrames
print("Original DataFrame:")
print(df_campaign_4.shape)

print("\nOversampled DataFrame:")
print(camp4_oversampled_train_df.shape)


Original DataFrame:
(60213, 249)

Oversampled DataFrame:
(75097, 249)


In [ ]:

# Separate majority and minority classes
majority_class = df_campaign_5[df_campaign_5['campaign_5'] == 0]
minority_class = df_campaign_5[df_campaign_5['campaign_5'] == 1]

# Calculate the number of minority samples needed for the target ratio
target_minority_count = int(len(majority_class) * target_ratio)

# Upsample minority class to reach the target ratio
minority_upsampled = resample(minority_class, replace=True, n_samples=target_minority_count, random_state=42)

# Combine the majority class with the upsampled minority class
camp5_oversampled_train_df = pd.concat([majority_class, minority_upsampled])

# Display the original and oversampled DataFrames
print("Original DataFrame:")
print(df_campaign_5.shape)

print("\nOversampled DataFrame:")
print(camp5_oversampled_train_df.shape)


Original DataFrame:
(60213, 249)

Oversampled DataFrame:
(72101, 249)


In [ ]:

# Separate majority and minority classes
majority_class = df_campaign_6[df_campaign_6['campaign_6'] == 0]
minority_class = df_campaign_6[df_campaign_6['campaign_6'] == 1]

# Calculate the number of minority samples needed for the target ratio
target_minority_count = int(len(majority_class) * target_ratio)

# Upsample minority class to reach the target ratio
minority_upsampled = resample(minority_class, replace=True, n_samples=target_minority_count, random_state=42)

# Combine the majority class with the upsampled minority class
camp6_oversampled_train_df = pd.concat([majority_class, minority_upsampled])

# Display the original and oversampled DataFrames
print("Original DataFrame:")
print(df_campaign_6.shape)

print("\nOversampled DataFrame:")
print(camp6_oversampled_train_df.shape)


Original DataFrame:
(60213, 249)

Oversampled DataFrame:
(75252, 249)


In [ ]:

# Separate majority and minority classes
majority_class = df_campaign_7[df_campaign_7['campaign_7'] == 0]
minority_class = df_campaign_7[df_campaign_7['campaign_7'] == 1]

# Calculate the number of minority samples needed for the target ratio
target_minority_count = int(len(majority_class) * target_ratio)

# Upsample minority class to reach the target ratio
minority_upsampled = resample(minority_class, replace=True, n_samples=target_minority_count, random_state=42)

# Combine the majority class with the upsampled minority class
camp7_oversampled_train_df = pd.concat([majority_class, minority_upsampled])

# Display the original and oversampled DataFrames
print("Original DataFrame:")
print(df_campaign_7.shape)

print("\nOversampled DataFrame:")
print(camp7_oversampled_train_df.shape)


Original DataFrame:
(60213, 249)

Oversampled DataFrame:
(75205, 249)


In [ ]:

# Separate majority and minority classes
majority_class = df_campaign_8[df_campaign_8['campaign_8'] == 0]
minority_class = df_campaign_8[df_campaign_8['campaign_8'] == 1]

# Calculate the number of minority samples needed for the target ratio
target_minority_count = int(len(majority_class) * target_ratio)

# Upsample minority class to reach the target ratio
minority_upsampled = resample(minority_class, replace=True, n_samples=target_minority_count, random_state=42)

# Combine the majority class with the upsampled minority class
camp8_oversampled_train_df = pd.concat([majority_class, minority_upsampled])

# Display the original and oversampled DataFrames
print("Original DataFrame:")
print(df_campaign_8.shape)

print("\nOversampled DataFrame:")
print(camp8_oversampled_train_df.shape)


Original DataFrame:
(60213, 249)

Oversampled DataFrame:
(74321, 249)


In [ ]:

# Separate majority and minority classes
majority_class = df_campaign_9[df_campaign_9['campaign_9'] == 0]
minority_class = df_campaign_9[df_campaign_9['campaign_9'] == 1]

# Calculate the number of minority samples needed for the target ratio
target_minority_count = int(len(majority_class) * target_ratio)

# Upsample minority class to reach the target ratio
minority_upsampled = resample(minority_class, replace=True, n_samples=target_minority_count, random_state=42)

# Combine the majority class with the upsampled minority class
camp9_oversampled_train_df = pd.concat([majority_class, minority_upsampled])

# Display the original and oversampled DataFrames
print("Original DataFrame:")
print(df_campaign_9.shape)

print("\nOversampled DataFrame:")
print(camp9_oversampled_train_df.shape)


Original DataFrame:
(60213, 249)

Oversampled DataFrame:
(75265, 249)


In [ ]:

# Separate majority and minority classes
majority_class = df_campaign_10[df_campaign_10['campaign_10'] == 0]
minority_class = df_campaign_10[df_campaign_10['campaign_10'] == 1]

# Calculate the number of minority samples needed for the target ratio
target_minority_count = int(len(majority_class) * target_ratio)

# Upsample minority class to reach the target ratio
minority_upsampled = resample(minority_class, replace=True, n_samples=target_minority_count, random_state=42)

# Combine the majority class with the upsampled minority class
camp10_oversampled_train_df = pd.concat([majority_class, minority_upsampled])

# Display the original and oversampled DataFrames
print("Original DataFrame:")
print(df_campaign_10.shape)

print("\nOversampled DataFrame:")
print(camp10_oversampled_train_df.shape)


Original DataFrame:
(60213, 249)

Oversampled DataFrame:
(75231, 249)


In [ ]:
#Problematic campaign
#df.campaign_11.value_counts()
#df.campaign_12.value_counts()
df.campaign_13.value_counts()
#df.campaign_14.value_counts()
#df.campaign_15.value_counts()
#df.campaign_16.value_counts()

0    60204
1        9
Name: campaign_13, dtype: int64

In [ ]:
df_campaign_1.head()

,allpresure2,Occupation_cat_D,ChurnPresure1,NumVouHybrid,PrepaidRevenue,NumberOfUpgrades,AddOnSubscriptionRevenue,NumMonPrepaid,CustomerSegment_cat_C,VouchersCount,MMSOnNet,OrderBill,Occupation_cat_Electrician,CustomerSegment_cat_Individual,Occupation_cat_Foreign Navy,DownFlux_PRev,bet,VASRevenue,Nationality,MaritalStatus_cat_Married,StartDate,deg_in,CustomerStatus_cat_Suspended,Occupation_cat_Accountant,NumberOfEnquiries,Occupation_cat_Architect,OtherRevenue,Occupation_cat_Specialist/Analyst/Advisor,HTM70CreditLimitBreach,SalesChannel_cat_Alternative,BilledMonth,TotalAmount_PostP,AmountOfCreditTransferTransactions,SalesChannel_cat_LMRA,CustomerSegment_cat_B,NumOrders,OrderWeek,CustomerSegment_cat_Medium SME,DaysToActiveStopDate,CDRBonusRevenueM_PoRev,Andpresure1,Occupation_cat_Police Officer,NumMMSPostpaid,Occupation_cat_Senior manager,RechargeForPrepaid,ETopUpCount,MigrationIN,NumSMS,NumberOfEnrollment,Occupation_cat_Military,PostpaidBalance,DataCharTraf,PostpaidDataRevenue,InterconnectPayableRevenue,eig_whole,RGE90Subscriberbase,CustomerSegment_cat_Large Enterprises,NumMMSPrepaid,ETopUp,TotalNETopUpCardCount,VoiceOnNet,CustomerTypeGroup_cat_Business Customer,ContractLengthD,CustomerSegment_cat_SME Flexible,PostpaidAddonDeActivations,OrderDay,iospresure2,NumVouPrepaid,DaysToSuspendStopDate,ChargeDurationM_PRev,TotalNETopUpCardAmount,eig_dir,ARPU,CustomerStatus_cat_Active,NumCalls,TotalMNPRequest,Occupation_cat_Dentist,LoyaltyPoints,Occupation_cat_A,Occupation_cat_Teacher,InterconnectReceivableRevenueRequest,NumberOfIncidents,CallDuration,allpresure1,OrderMonth,ChurnPresure2,CieledMoUM_PoRev,CustomerTypeGroup_cat_Individual,Occupation_cat_Nurse/Other medical staff,NumMGRPrepaid,Occupation_cat_Security/Guard,Occupation_cat_B,NumVouPostpaid,UpFlux_PoRev,NoOfEvents,NumSMSPrepaid,Occupation_cat_Sales Representative,UpFlux_PRev,Occupation_cat_Labour Worker,HTM100CreditLimitBreach,Occupation_cat_Labour worker,Gender_cat_Male,DownFlux_PoRev,DataRevenue,Occupation_cat_CEO,TotalNRechargesUpto2,BirthDate,ChargeFromPostpaid,SMSOnNet,ChargeFromPrepaid,VoiceRevenue,SalesChannel_cat_SalesChannel_Unknown,SMSRevenue,SubscriberID,Occupation_cat_Engineer,NumberOfTroubleTickets,PostpaidBalanceBefore,RechargeValue,GlobalVouchers,Occupation_cat_Mechanic,PackagePlanSubscriptionDate,CustomerStatus_cat_Inactive,NumberOfServiceRequests,TotalNRechargesUpto3,TotalAmount_PrePa,OrderExp,RGE30Subscriberbase,EoP_SubscribersApOG,NumberOfTicketsOpenedToday,CustomerSegment_cat_Contractor,VoiceOffNet,ActualMoUM_PoRev,SnapshotDate,MMSInt,TotalFlux,NumRechargeMine,CustomerSegment_cat_Government,MaritalStatus_cat_Single,NumVoicePrepaid,MMSLength,Occupation_cat_C,PrepaidBalance,NumberOfTermination,RoamingOutboundRevenue,Occupation_cat_Manager,SalesChannel_cat_UniConnect,OrderNULL,CustomerSegment_cat_Powered MEs,ChargeAmount,RoamingOutound,Occupation_cat_Senior Manager,EoPProvisionedBase,TotalNRechargesUpto250,CycleDetails,CustomerSegment_cat_Employees,MMSRevenue,postpaidAddonRevenue,NumDataSPrepaid,CustomerSegment_cat_D,NumRecharges,RechargeMonth,Occupation_cat_Banker,Occupation_cat_Lawyer,NumMonNULL,ContractLengthM,CustomerStatus_cat_New,SessionsLength,NoEventsVASPost,Occupation_cat_Consultant,PostpaidMMSRev,CustomerSegment_cat_Test,TotalNRechargesUpto5,PromiseToPay,CieledMoUM_PRev,Occupation_cat_Secretary,Andpresure2,CustomerSegment_cat_Special,Occupation_cat_Biologist/Chemist/Physician,ChargeDurationM_PoRev,Vouchers,deg_out,NumSMSPostpaid,TotalNRechargesUpto05,PrepaidBalanceBefore,LevelPoints,PostpaidVoiceRevenue,CustomerSegment_cat_Small SME,HTM90CreditLimitBreach,ExpiredPoints,Gender_cat_Female,VoiceInt,NumVoicePostpaid,IsCommitment,NumMMS,TotalNRechargesUpto25,Occupation_cat_Doctor,Occupation_cat_Supervisor,NumDataPostpaid,Occupation_cat_Pilot,EndDate,Occupation_cat_Student,PostpaidRevenues,ReadeemablePoints,MMSOffNet,Occupation_cat_Office Clerk/Office worker,Occupation_cat_Technician,eig_simple,SMSLength,NumberOfDowngrades,NumberOfCreditTransferTransactions

In [ ]:
#camp10_oversampled_train_df
X_campaign_1 = camp1_oversampled_train_df.drop(columns=['campaign_1']) #features (independent variables)
y_campaign_1 = camp1_oversampled_train_df['campaign_1'] #target (dependent variable)
#############
X_campaign_2 = camp2_oversampled_train_df.drop(columns=['campaign_2']) #features (independent variables)
y_campaign_2 = camp2_oversampled_train_df['campaign_2'] #target (dependent variable)
#############
X_campaign_3 = camp3_oversampled_train_df.drop(columns=['campaign_3']) #features (independent variables)
y_campaign_3 = camp3_oversampled_train_df['campaign_3'] #target (dependent variable)
#############
X_campaign_4 = camp4_oversampled_train_df.drop(columns=['campaign_4']) #features (independent variables)
y_campaign_4 = camp4_oversampled_train_df['campaign_4'] #target (dependent variable)
#############
X_campaign_5 = camp5_oversampled_train_df.drop(columns=['campaign_5']) #features (independent variables)
y_campaign_5 = camp5_oversampled_train_df['campaign_5'] #target (dependent variable)
#############
X_campaign_6 = camp6_oversampled_train_df.drop(columns=['campaign_6']) #features (independent variables)
y_campaign_6 = camp6_oversampled_train_df['campaign_6'] #target (dependent variable)
#############
X_campaign_7 = camp7_oversampled_train_df.drop(columns=['campaign_7']) #features (independent variables)
y_campaign_7 = camp7_oversampled_train_df['campaign_7'] #target (dependent variable)
#############
X_campaign_8 = camp8_oversampled_train_df.drop(columns=['campaign_8']) #features (independent variables)
y_campaign_8 = camp8_oversampled_train_df['campaign_8'] #target (dependent variable)
#############
X_campaign_9 = camp9_oversampled_train_df.drop(columns=['campaign_9']) #features (independent variables)
y_campaign_9 = camp9_oversampled_train_df['campaign_9'] #target (dependent variable)
#############
X_campaign_10 = camp10_oversampled_train_df.drop(columns=['campaign_10']) #features (independent variables)
y_campaign_10 = camp10_oversampled_train_df['campaign_10'] #target (dependent variable)


In [ ]:
X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test = train_test_split(X_campaign_1, y_campaign_1, test_size = 0.2, random_state=42)
X_cap2_train, X_cap2_test, y_cap2_train, y_cap2_test = train_test_split(X_campaign_2, y_campaign_2, test_size = 0.2, random_state=42)
X_cap3_train, X_cap3_test, y_cap3_train, y_cap3_test = train_test_split(X_campaign_3, y_campaign_3, test_size = 0.2, random_state=42)
X_cap4_train, X_cap4_test, y_cap4_train, y_cap4_test = train_test_split(X_campaign_4, y_campaign_4, test_size = 0.2, random_state=42)
X_cap5_train, X_cap5_test, y_cap5_train, y_cap5_test = train_test_split(X_campaign_5, y_campaign_5, test_size = 0.2, random_state=42)
X_cap6_train, X_cap6_test, y_cap6_train, y_cap6_test = train_test_split(X_campaign_6, y_campaign_6, test_size = 0.2, random_state=42)
X_cap7_train, X_cap7_test, y_cap7_train, y_cap7_test = train_test_split(X_campaign_7, y_campaign_7, test_size = 0.2, random_state=42)
X_cap8_train, X_cap8_test, y_cap8_train, y_cap8_test = train_test_split(X_campaign_8, y_campaign_8, test_size = 0.2, random_state=42)
X_cap9_train, X_cap9_test, y_cap9_train, y_cap9_test = train_test_split(X_campaign_9, y_campaign_9, test_size = 0.2, random_state=42)
X_cap10_train, X_cap10_test, y_cap10_train, y_cap10_test = train_test_split(X_campaign_10, y_campaign_10, test_size = 0.2, random_state=42)


In [ ]:
# Creating an empty DataFrame for campaign 1
result_df_1 = pd.DataFrame(columns=['SubscriberID', 'actual_value'])
result_df_1['SubscriberID'] = X_cap1_test['SubscriberID'].copy()
result_df_1['actual_value'] = y_cap1_test
#########################################
# Creating an empty DataFrame for campaign 2
result_df_2 = pd.DataFrame(columns=['SubscriberID', 'actual_value'])
result_df_2['SubscriberID'] = X_cap2_test['SubscriberID'].copy()
result_df_2['actual_value'] = y_cap2_test
#########################################
# Creating an empty DataFrame for campaign 3
result_df_3 = pd.DataFrame(columns=['SubscriberID', 'actual_value'])
result_df_3['SubscriberID'] = X_cap3_test['SubscriberID'].copy()
result_df_3['actual_value'] = y_cap3_test
#########################################
# Creating an empty DataFrame for campaign 4
result_df_4 = pd.DataFrame(columns=['SubscriberID', 'actual_value'])
result_df_4['SubscriberID'] = X_cap4_test['SubscriberID'].copy()
result_df_4['actual_value'] = y_cap4_test
#########################################
# Creating an empty DataFrame for campaign 5
result_df_5 = pd.DataFrame(columns=['SubscriberID', 'actual_value'])
result_df_5['SubscriberID'] = X_cap5_test['SubscriberID'].copy()
result_df_5['actual_value'] = y_cap5_test
#########################################
# Creating an empty DataFrame for campaign 6
result_df_6 = pd.DataFrame(columns=['SubscriberID', 'actual_value'])
result_df_6['SubscriberID'] = X_cap6_test['SubscriberID'].copy()
result_df_6['actual_value'] = y_cap6_test
#########################################
# Creating an empty DataFrame for campaign 7
result_df_7 = pd.DataFrame(columns=['SubscriberID', 'actual_value'])
result_df_7['SubscriberID'] = X_cap7_test['SubscriberID'].copy()
result_df_7['actual_value'] = y_cap7_test
#########################################
# Creating an empty DataFrame for campaign 8
result_df_8 = pd.DataFrame(columns=['SubscriberID', 'actual_value'])
result_df_8['SubscriberID'] = X_cap8_test['SubscriberID'].copy()
result_df_8['actual_value'] = y_cap8_test
#########################################
# Creating an empty DataFrame for campaign 9
result_df_9 = pd.DataFrame(columns=['SubscriberID', 'actual_value'])
result_df_9['SubscriberID'] = X_cap9_test['SubscriberID'].copy()
result_df_9['actual_value'] = y_cap9_test
#########################################
# Creating an empty DataFrame for campaign 10
result_df_10 = pd.DataFrame(columns=['SubscriberID', 'actual_value'])
result_df_10['SubscriberID'] = X_cap10_test['SubscriberID'].copy()
result_df_10['actual_value'] = y_cap10_test
#########################################

In [ ]:
# Method for correlation control
def drop_highly_correlated_columns(df, threshold):
    # Create correlation matrix
    corr_matrix = df.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find features with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(upper[column] > float(threshold))]

    # Drop features
    df.drop(to_drop, axis=1, inplace=True)
    return df  # Return None if no high correlation is found

In [ ]:
%%time
threshold = 0.1
X_cap1_train = drop_highly_correlated_columns(X_cap1_train, threshold)
print('For campain 1 => ', len(X_cap1_train.columns))
#####################################################
X_cap2_train = drop_highly_correlated_columns(X_cap2_train, threshold)
print('For campain 2 => ', len(X_cap2_train.columns))
#####################################################
X_cap3_train = drop_highly_correlated_columns(X_cap3_train, threshold)
print('For campain 3 => ', len(X_cap3_train.columns))
#####################################################
X_cap4_train = drop_highly_correlated_columns(X_cap4_train, threshold)
print('For campain 4 => ', len(X_cap4_train.columns))
#####################################################
X_cap5_train = drop_highly_correlated_columns(X_cap5_train, threshold)
print('For campain 5 => ', len(X_cap5_train.columns))
#####################################################
X_cap6_train = drop_highly_correlated_columns(X_cap6_train, threshold)
print('For campain 6 => ', len(X_cap6_train.columns))
#####################################################
X_cap7_train = drop_highly_correlated_columns(X_cap7_train, threshold)
print('For campain 7 => ', len(X_cap7_train.columns))
#####################################################
X_cap8_train = drop_highly_correlated_columns(X_cap8_train, threshold)
print('For campain 8 => ', len(X_cap8_train.columns))
#####################################################
X_cap9_train = drop_highly_correlated_columns(X_cap9_train, threshold)
print('For campain 9 => ', len(X_cap9_train.columns))
#####################################################
X_cap10_train = drop_highly_correlated_columns(X_cap10_train, threshold)
print('For campain 10 => ', len(X_cap10_train.columns))
#####################################################

For campain 1 =>  97
For campain 2 =>  94
For campain 3 =>  96
For campain 4 =>  94
For campain 5 =>  96
For campain 6 =>  95
For campain 7 =>  92
For campain 8 =>  94
For campain 9 =>  91
For campain 10 =>  87
CPU times: user 1min 54s, sys: 551 ms, total: 1min 55s
Wall time: 1min 58s


In [ ]:
X_cap8_train.head().append(X_cap8_train.tail())

In [ ]:
X_cap10_train

In [ ]:
X_cap1_train.shape, X_cap5_train.shape, X_cap9_train.shape, X_cap10_train.shape

((59304, 97), (57680, 96), (60212, 91), (60184, 87))

In [ ]:
corr_col_list_1 = X_cap1_train.columns.tolist()
X_cap1_train = X_cap1_train[corr_col_list_1].copy()
X_cap1_test = X_cap1_test[corr_col_list_1].copy()
print(X_cap1_train.shape, X_cap1_test.shape)
############################################
corr_col_list_2 = X_cap2_train.columns.tolist()
X_cap2_train = X_cap2_train[corr_col_list_2].copy()
X_cap2_test = X_cap2_test[corr_col_list_2].copy()
############################################
corr_col_list_3 = X_cap3_train.columns.tolist()
X_cap3_train = X_cap3_train[corr_col_list_3].copy()
X_cap3_test = X_cap3_test[corr_col_list_3].copy()
############################################
corr_col_list_4 = X_cap4_train.columns.tolist()
X_cap4_train = X_cap4_train[corr_col_list_4].copy()
X_cap4_test = X_cap4_test[corr_col_list_4].copy()
############################################
corr_col_list_5 = X_cap5_train.columns.tolist()
X_cap5_train = X_cap5_train[corr_col_list_5].copy()
X_cap5_test = X_cap5_test[corr_col_list_5].copy()
############################################
corr_col_list_6 = X_cap6_train.columns.tolist()
X_cap6_train = X_cap6_train[corr_col_list_6].copy()
X_cap6_test = X_cap6_test[corr_col_list_6].copy()
############################################
corr_col_list_7 = X_cap7_train.columns.tolist()
X_cap7_train = X_cap7_train[corr_col_list_7].copy()
X_cap7_test = X_cap7_test[corr_col_list_7].copy()
############################################
corr_col_list_8 = X_cap8_train.columns.tolist()
X_cap8_train = X_cap8_train[corr_col_list_8].copy()
X_cap8_test = X_cap8_test[corr_col_list_8].copy()
############################################
corr_col_list_9 = X_cap9_train.columns.tolist()
X_cap9_train = X_cap9_train[corr_col_list_9].copy()
X_cap9_test = X_cap9_test[corr_col_list_9].copy()
############################################
corr_col_list_10 = X_cap10_train.columns.tolist()
X_cap10_train = X_cap10_train[corr_col_list_10].copy()
X_cap10_test = X_cap10_test[corr_col_list_10].copy()
############################################

(59304, 97) (14826, 97)


##***Machine Learning Models***

## Random Forest Classfier

In [ ]:
def random_forest_classifier(x_train, y_train, filename):
    print("random_forest_classifier started")

    hyper_random = {"n_estimators": [300, 700],
                    "max_depth": [10, 8],
                    "min_samples_split": [10, 5],
                    "min_samples_leaf": [5, 3]}

    clf_rf_tuned = GridSearchCV(RandomForestClassifier(), hyper_random,
                                cv=5, verbose=1,
                                n_jobs=-1)
    clf_rf_tuned.fit(x_train, y_train)

    best_params_random = clf_rf_tuned.best_params_
    print("best_params_random = ", best_params_random)

    cv_clf_rf = RandomForestClassifier(max_depth=best_params_random["max_depth"],
                                       min_samples_leaf=best_params_random["min_samples_leaf"],
                                       min_samples_split=best_params_random["min_samples_split"],
                                       n_estimators=best_params_random["n_estimators"])

    cv_clf_rf.fit(x_train, y_train)

    #Dump model
    # save the model to disk
    pickle.dump(cv_clf_rf, open(filename, 'wb'))
    print(filename, ' dumped !!!')

    return cv_clf_rf

In [ ]:
%%time
CV_clf_rf_1 = random_forest_classifier(X_cap1_train, y_cap1_train, 'random_forest_classifier_1.sav')


random_forest_classifier started
Fitting 5 folds for each of 16 candidates, totalling 80 fits
best_params_random =  {'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 700}
random_forest_classifier_1.sav  dumped !!!
CPU times: user 1min 4s, sys: 2.73 s, total: 1min 7s
Wall time: 18min 29s


In [127]:
#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_random_camp1 = CV_clf_rf_1.predict_proba(X_cap1_test)[:, 1]
yhat_random_camp1 = CV_clf_rf_1.predict(X_cap1_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap1_test, y_test_predict_random_camp1, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap1_test, yhat_random_camp1)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap1_test, yhat_random_camp1))

Accuracy: 0.8051396195872117
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11892
           1       0.94      0.02      0.03      2934

    accuracy                           0.81     14826
   macro avg       0.87      0.51      0.46     14826
weighted avg       0.83      0.81      0.72     14826



In [ ]:
%%time
CV_clf_rf_2 = random_forest_classifier(X_cap2_train, y_cap2_train, 'random_forest_classifier_2.sav')

random_forest_classifier started
Fitting 5 folds for each of 16 candidates, totalling 80 fits
best_params_random =  {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 10, 'n_estimators': 300}
random_forest_classifier_2.sav  dumped !!!
CPU times: user 34.8 s, sys: 2.7 s, total: 37.5 s
Wall time: 17min 47s


In [128]:
#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_random_camp2 = CV_clf_rf_2.predict_proba(X_cap2_test)[:, 1]
yhat_random_camp2 = CV_clf_rf_2.predict(X_cap2_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap2_test, y_test_predict_random_camp2, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap2_test, yhat_random_camp2)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap2_test, yhat_random_camp2))

Accuracy: 0.8020625644551392
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11666
           1       0.00      0.00      0.00      2879

    accuracy                           0.80     14545
   macro avg       0.40      0.50      0.45     14545
weighted avg       0.64      0.80      0.71     14545



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
%%time
CV_clf_rf_3 = random_forest_classifier(X_cap3_train, y_cap3_train, 'random_forest_classifier_3.sav')

random_forest_classifier started
Fitting 5 folds for each of 16 candidates, totalling 80 fits
best_params_random =  {'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 700}
random_forest_classifier_3.sav  dumped !!!
CPU times: user 1min 2s, sys: 2.38 s, total: 1min 4s
Wall time: 16min 48s


In [129]:
#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_random_camp3 = CV_clf_rf_3.predict_proba(X_cap3_test)[:, 1]
yhat_random_camp3 = CV_clf_rf_3.predict(X_cap3_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap3_test, y_test_predict_random_camp3, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap3_test, yhat_random_camp3)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap3_test, yhat_random_camp3))

Accuracy: 0.8029718931800659
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11456
           1       0.00      0.00      0.00      2811

    accuracy                           0.80     14267
   macro avg       0.40      0.50      0.45     14267
weighted avg       0.64      0.80      0.72     14267



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
%%time
CV_clf_rf_4 = random_forest_classifier(X_cap4_train, y_cap4_train, 'random_forest_classifier_4.sav')

random_forest_classifier started
Fitting 5 folds for each of 16 candidates, totalling 80 fits
best_params_random =  {'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 300}
random_forest_classifier_4.sav  dumped !!!
CPU times: user 31.5 s, sys: 2.07 s, total: 33.5 s
Wall time: 15min 10s


In [130]:
#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_random_camp4 = CV_clf_rf_4.predict_proba(X_cap4_test)[:, 1]
yhat_random_camp4 = CV_clf_rf_4.predict(X_cap4_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap4_test, y_test_predict_random_camp4, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap4_test, yhat_random_camp4)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap4_test, yhat_random_camp4))

Accuracy: 0.8065912117177098
              precision    recall  f1-score   support

           0       0.81      1.00      0.89     12069
           1       1.00      0.02      0.03      2951

    accuracy                           0.81     15020
   macro avg       0.90      0.51      0.46     15020
weighted avg       0.84      0.81      0.72     15020



In [ ]:
%%time
CV_clf_rf_5 = random_forest_classifier(X_cap5_train, y_cap5_train, 'random_forest_classifier_5.sav')

random_forest_classifier started
Fitting 5 folds for each of 16 candidates, totalling 80 fits
best_params_random =  {'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 300}
random_forest_classifier_5.sav  dumped !!!
CPU times: user 33.6 s, sys: 2.32 s, total: 35.9 s
Wall time: 16min 12s


In [131]:
#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_random_camp5 = CV_clf_rf_5.predict_proba(X_cap5_test)[:, 1]
yhat_random_camp5 = CV_clf_rf_5.predict(X_cap5_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap5_test, y_test_predict_random_camp5, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap5_test, yhat_random_camp5)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap5_test, yhat_random_camp5))

Accuracy: 0.8032036613272311
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11580
           1       1.00      0.00      0.00      2841

    accuracy                           0.80     14421
   macro avg       0.90      0.50      0.45     14421
weighted avg       0.84      0.80      0.72     14421



In [ ]:
%%time
CV_clf_rf_6 = random_forest_classifier(X_cap6_train, y_cap6_train, 'random_forest_classifier_6.sav')

random_forest_classifier started
Fitting 5 folds for each of 16 candidates, totalling 80 fits
best_params_random =  {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 300}
random_forest_classifier_6.sav  dumped !!!
CPU times: user 24.3 s, sys: 1.72 s, total: 26 s
Wall time: 12min 40s


In [132]:
#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_random_camp6 = CV_clf_rf_6.predict_proba(X_cap6_test)[:, 1]
yhat_random_camp6 = CV_clf_rf_6.predict(X_cap6_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap6_test, y_test_predict_random_camp6, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap6_test, yhat_random_camp6)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap6_test, yhat_random_camp6))

Accuracy: 0.9773436980931499
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     12079
           1       0.97      0.91      0.94      2972

    accuracy                           0.98     15051
   macro avg       0.98      0.95      0.96     15051
weighted avg       0.98      0.98      0.98     15051



In [ ]:
%%time
CV_clf_rf_7 = random_forest_classifier(X_cap7_train, y_cap7_train, 'random_forest_classifier_7.sav')

random_forest_classifier started
Fitting 5 folds for each of 16 candidates, totalling 80 fits
best_params_random =  {'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 5, 'n_estimators': 300}
random_forest_classifier_7.sav  dumped !!!
CPU times: user 26.4 s, sys: 1.67 s, total: 28 s
Wall time: 14min


In [133]:
#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_random_camp7 = CV_clf_rf_7.predict_proba(X_cap7_test)[:, 1]
yhat_random_camp7 = CV_clf_rf_7.predict(X_cap7_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap7_test, y_test_predict_random_camp7, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap7_test, yhat_random_camp7)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap7_test, yhat_random_camp7))

Accuracy: 0.8627085965028921
              precision    recall  f1-score   support

           0       0.87      0.97      0.92     12100
           1       0.78      0.42      0.54      2941

    accuracy                           0.86     15041
   macro avg       0.82      0.69      0.73     15041
weighted avg       0.85      0.86      0.85     15041



In [ ]:
%%time
CV_clf_rf_8 = random_forest_classifier(X_cap8_train, y_cap8_train, 'random_forest_classifier_8.sav')

random_forest_classifier started
Fitting 5 folds for each of 16 candidates, totalling 80 fits
best_params_random =  {'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 5, 'n_estimators': 700}
random_forest_classifier_8.sav  dumped !!!
CPU times: user 57.6 s, sys: 2.18 s, total: 59.8 s
Wall time: 16min 49s


In [134]:
#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_random_camp8 = CV_clf_rf_8.predict_proba(X_cap8_test)[:, 1]
yhat_random_camp8 = CV_clf_rf_8.predict(X_cap8_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap8_test, y_test_predict_random_camp8, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap8_test, yhat_random_camp8)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap8_test, yhat_random_camp8))

Accuracy: 0.8010090817356206
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11893
           1       1.00      0.00      0.01      2972

    accuracy                           0.80     14865
   macro avg       0.90      0.50      0.45     14865
weighted avg       0.84      0.80      0.71     14865



In [ ]:
%%time
CV_clf_rf_9 = random_forest_classifier(X_cap9_train, y_cap9_train, 'random_forest_classifier_9.sav')

random_forest_classifier started
Fitting 5 folds for each of 16 candidates, totalling 80 fits
best_params_random =  {'max_depth': 8, 'min_samples_leaf': 5, 'min_samples_split': 10, 'n_estimators': 300}
random_forest_classifier_9.sav  dumped !!!
CPU times: user 19.8 s, sys: 1.51 s, total: 21.3 s
Wall time: 11min


In [135]:
#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_random_camp9 = CV_clf_rf_9.predict_proba(X_cap9_test)[:, 1]
yhat_random_camp9 = CV_clf_rf_9.predict(X_cap9_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap9_test, y_test_predict_random_camp9, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap9_test, yhat_random_camp9)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap9_test, yhat_random_camp9))

Accuracy: 0.9934232378927789
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     12046
           1       0.97      1.00      0.98      3007

    accuracy                           0.99     15053
   macro avg       0.98      1.00      0.99     15053
weighted avg       0.99      0.99      0.99     15053



In [ ]:
%%time
CV_clf_rf_10 = random_forest_classifier(X_cap10_train, y_cap10_train, 'random_forest_classifier_10.sav')

In [136]:
#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_random_camp10 = CV_clf_rf_10.predict_proba(X_cap10_test)[:, 1]
yhat_random_camp10 = CV_clf_rf_10.predict(X_cap10_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap10_test, y_test_predict_random_camp10, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap10_test, yhat_random_camp10)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap10_test, yhat_random_camp10))

Accuracy: 0.8367116368711371
              precision    recall  f1-score   support

           0       0.83      1.00      0.91     12045
           1       1.00      0.18      0.31      3002

    accuracy                           0.84     15047
   macro avg       0.91      0.59      0.61     15047
weighted avg       0.86      0.84      0.79     15047



In [142]:
X_prediction_camp1 = X[corr_col_list_1].copy()
X_prediction_camp2 = X[corr_col_list_2].copy()
X_prediction_camp3 = X[corr_col_list_3].copy()
X_prediction_camp4 = X[corr_col_list_4].copy()
X_prediction_camp5 = X[corr_col_list_5].copy()
X_prediction_camp6 = X[corr_col_list_6].copy()
X_prediction_camp7 = X[corr_col_list_7].copy()
X_prediction_camp8 = X[corr_col_list_8].copy()
X_prediction_camp9 = X[corr_col_list_9].copy()
X_prediction_camp10 = X[corr_col_list_10].copy()

In [143]:
%%time
rf_results_camp1 = CV_clf_rf_1.predict_proba(X_prediction_camp1)[:, 1]
rf_results_camp2 = CV_clf_rf_2.predict_proba(X_prediction_camp2)[:, 1]
rf_results_camp3 = CV_clf_rf_3.predict_proba(X_prediction_camp3)[:, 1]
rf_results_camp4 = CV_clf_rf_4.predict_proba(X_prediction_camp4)[:, 1]
rf_results_camp5 = CV_clf_rf_5.predict_proba(X_prediction_camp5)[:, 1]
rf_results_camp6 = CV_clf_rf_6.predict_proba(X_prediction_camp6)[:, 1]
rf_results_camp7 = CV_clf_rf_7.predict_proba(X_prediction_camp7)[:, 1]
rf_results_camp8 = CV_clf_rf_8.predict_proba(X_prediction_camp8)[:, 1]
rf_results_camp9 = CV_clf_rf_9.predict_proba(X_prediction_camp9)[:, 1]
rf_results_camp10 = CV_clf_rf_10.predict_proba(X_prediction_camp10)[:, 1]


CPU times: user 29.3 s, sys: 21.6 ms, total: 29.3 s
Wall time: 30.4 s


In [145]:
final_results_rf = pd.DataFrame(columns = ['SubscriberID', 'prob_results_rf_1'])
final_results_rf['SubscriberID'] = X['SubscriberID'].copy()
final_results_rf['prob_results_rf_1'] = rf_results_camp1
final_results_rf['prob_results_rf_2'] = rf_results_camp2
final_results_rf['prob_results_rf_3'] = rf_results_camp3
final_results_rf['prob_results_rf_4'] = rf_results_camp4
final_results_rf['prob_results_rf_5'] = rf_results_camp5
final_results_rf['prob_results_rf_6'] = rf_results_camp6
final_results_rf['prob_results_rf_7'] = rf_results_camp7
final_results_rf['prob_results_rf_8'] = rf_results_camp8
final_results_rf['prob_results_rf_9'] = rf_results_camp9
final_results_rf['prob_results_rf_10'] = rf_results_camp10
final_results_rf.head()

,SubscriberID,prob_results_rf_1,prob_results_rf_2,prob_results_rf_3,prob_results_rf_4,prob_results_rf_5,prob_results_rf_6,prob_results_rf_7,prob_results_rf_8,prob_results_rf_9,prob_results_rf_10
1,4629739439,0.259914,0.217504,0.185521,0.238323,0.202069,0.067267,0.170452,0.190467,0.036856,0.163187
2,10505531439,0.137936,0.251644,0.200538,0.315057,0.210328,0.056868,0.072705,0.213112,0.062082,0.093859
3,12309121439,0.166742,0.192241,0.210715,0.120472,0.231834,0.070246,0.131997,0.189260,0.018200,0.068898
4,13634651439,0.177301,0.215344,0.200381,0.168308,0.211474,0.100343,0.145285,0.191114,0.115662,0.179885
5,13634651439,0.177235,0.215426,0.200238,0.184674,0.211443,0.104338,0.145285,0.191082,0.115662,0.175266


In [150]:
final_results_rf.to_csv('rf_final_results.csv', index=False)

In [152]:
rf_final_results = pd.read_csv('rf_final_results.csv')
rf_final_results.head()

,SubscriberID,prob_results_rf_1,prob_results_rf_2,prob_results_rf_3,prob_results_rf_4,prob_results_rf_5,prob_results_rf_6,prob_results_rf_7,prob_results_rf_8,prob_results_rf_9,prob_results_rf_10
0,4629739439,0.259914,0.217504,0.185521,0.238323,0.202069,0.067267,0.170452,0.190467,0.036856,0.163187
1,10505531439,0.137936,0.251644,0.200538,0.315057,0.210328,0.056868,0.072705,0.213112,0.062082,0.093859
2,12309121439,0.166742,0.192241,0.210715,0.120472,0.231834,0.070246,0.131997,0.189260,0.018200,0.068898
3,13634651439,0.177301,0.215344,0.200381,0.168308,0.211474,0.100343,0.145285,0.191114,0.115662,0.179885
4,13634651439,0.177235,0.215426,0.200238,0.184674,0.211443,0.104338,0.145285,0.191082,0.115662,0.175266


## Calibrated Random Forest Classfier

In [154]:
def calibrated_classifier(x_train, y_train, cv_clf_rf, filename):
    print("calibrated_classifier started")

    # Create a corrected classifier.
    clf_sigmoid = CalibratedClassifierCV(cv_clf_rf, cv=10, method='sigmoid')
    clf_sigmoid.fit(x_train, y_train)

    #Dump model
    # save the model to disk
    pickle.dump(clf_sigmoid, open(filename, 'wb'))
    print(filename, ' dumped !!!')
    return clf_sigmoid

In [155]:
%%time
CV_clf_sigmoid_rf_1 = calibrated_classifier(X_cap1_train, y_cap1_train, CV_clf_rf_1, 'calib_random_forest_classifier_1.sav')

calibrated_classifier started
calib_random_forest_classifier_1.sav  dumped !!!
CPU times: user 3min 59s, sys: 378 ms, total: 3min 59s
Wall time: 4min 25s


In [156]:
#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_cal_random_camp1 = CV_clf_sigmoid_rf_1.predict_proba(X_cap1_test)[:, 1]
yhat_cal_random_camp1 = CV_clf_sigmoid_rf_1.predict(X_cap1_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap1_test, y_test_predict_cal_random_camp1, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap1_test, yhat_cal_random_camp1)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap1_test, yhat_cal_random_camp1))

Accuracy: 0.8168083097261567
              precision    recall  f1-score   support

           0       0.82      0.98      0.90     11892
           1       0.69      0.14      0.23      2934

    accuracy                           0.82     14826
   macro avg       0.75      0.56      0.56     14826
weighted avg       0.80      0.82      0.76     14826



In [157]:
%%time
CV_clf_sigmoid_rf_2 = calibrated_classifier(X_cap2_train, y_cap2_train, CV_clf_rf_2, 'calib_random_forest_classifier_2.sav')

calibrated_classifier started
calib_random_forest_classifier_2.sav  dumped !!!
CPU times: user 1min 50s, sys: 259 ms, total: 1min 50s
Wall time: 2min 30s


In [158]:
#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_cal_random_camp2 = CV_clf_sigmoid_rf_2.predict_proba(X_cap2_test)[:, 1]
yhat_cal_random_camp2 = CV_clf_sigmoid_rf_2.predict(X_cap2_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap2_test, y_test_predict_cal_random_camp2, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap2_test, yhat_cal_random_camp2)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap2_test, yhat_cal_random_camp2))

Accuracy: 0.8020625644551392
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11666
           1       0.50      0.02      0.04      2879

    accuracy                           0.80     14545
   macro avg       0.65      0.51      0.46     14545
weighted avg       0.74      0.80      0.72     14545



In [159]:
%%time
CV_clf_sigmoid_rf_3 = calibrated_classifier(X_cap3_train, y_cap3_train, CV_clf_rf_3, 'calib_random_forest_classifier_3.sav')

calibrated_classifier started
calib_random_forest_classifier_3.sav  dumped !!!
CPU times: user 1min 41s, sys: 213 ms, total: 1min 41s
Wall time: 2min 4s


In [160]:
#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_cal_random_camp3 = CV_clf_sigmoid_rf_3.predict_proba(X_cap3_test)[:, 1]
yhat_cal_random_camp3 = CV_clf_sigmoid_rf_3.predict(X_cap3_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap3_test, y_test_predict_cal_random_camp3, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap3_test, yhat_cal_random_camp3)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap3_test, yhat_cal_random_camp3))

Accuracy: 0.804093362304619
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11456
           1       0.64      0.01      0.03      2811

    accuracy                           0.80     14267
   macro avg       0.72      0.51      0.46     14267
weighted avg       0.77      0.80      0.72     14267



In [161]:
%%time
CV_clf_sigmoid_rf_4 = calibrated_classifier(X_cap4_train, y_cap4_train, CV_clf_rf_4, 'calib_random_forest_classifier_4.sav')

#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_cal_random_camp4 = CV_clf_sigmoid_rf_4.predict_proba(X_cap4_test)[:, 1]
yhat_cal_random_camp4 = CV_clf_sigmoid_rf_4.predict(X_cap4_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap4_test, y_test_predict_cal_random_camp4, n_bins=10)

# Evaluate the accuracy
accuracy = accuracy_score(y_cap4_test, yhat_cal_random_camp4)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap4_test, yhat_cal_random_camp4))

calibrated_classifier started
calib_random_forest_classifier_4.sav  dumped !!!
Accuracy: 0.8280292942743009
              precision    recall  f1-score   support

           0       0.85      0.95      0.90     12069
           1       0.62      0.32      0.43      2951

    accuracy                           0.83     15020
   macro avg       0.74      0.64      0.66     15020
weighted avg       0.81      0.83      0.81     15020

CPU times: user 3min 50s, sys: 445 ms, total: 3min 51s
Wall time: 4min 23s


In [162]:
%%time
CV_clf_sigmoid_rf_5 = calibrated_classifier(X_cap5_train, y_cap5_train, CV_clf_rf_5, 'calib_random_forest_classifier_5.sav')

#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_cal_random_camp5 = CV_clf_sigmoid_rf_5.predict_proba(X_cap5_test)[:, 1]
yhat_cal_random_camp5 = CV_clf_sigmoid_rf_5.predict(X_cap5_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap5_test, y_test_predict_cal_random_camp5, n_bins=10)

# Evaluate the accuracy
accuracy = accuracy_score(y_cap5_test, yhat_cal_random_camp5)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap5_test, yhat_cal_random_camp5))

calibrated_classifier started
calib_random_forest_classifier_5.sav  dumped !!!
Accuracy: 0.8029956313709175
              precision    recall  f1-score   support

           0       0.81      0.99      0.89     11580
           1       0.50      0.03      0.06      2841

    accuracy                           0.80     14421
   macro avg       0.65      0.51      0.48     14421
weighted avg       0.75      0.80      0.73     14421

CPU times: user 3min 39s, sys: 447 ms, total: 3min 39s
Wall time: 3min 41s


In [163]:
%%time
CV_clf_sigmoid_rf_6 = calibrated_classifier(X_cap6_train, y_cap6_train, CV_clf_rf_6, 'calib_random_forest_classifier_6.sav')

#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_cal_random_camp6 = CV_clf_sigmoid_rf_6.predict_proba(X_cap6_test)[:, 1]
yhat_cal_random_camp6 = CV_clf_sigmoid_rf_6.predict(X_cap6_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap6_test, y_test_predict_cal_random_camp6, n_bins=10)

# Evaluate the accuracy
accuracy = accuracy_score(y_cap6_test, yhat_cal_random_camp6)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap6_test, yhat_cal_random_camp6))

calibrated_classifier started
calib_random_forest_classifier_6.sav  dumped !!!
Accuracy: 0.9673111421168029
              precision    recall  f1-score   support

           0       0.98      0.98      0.98     12079
           1       0.92      0.91      0.92      2972

    accuracy                           0.97     15051
   macro avg       0.95      0.95      0.95     15051
weighted avg       0.97      0.97      0.97     15051

CPU times: user 3min 23s, sys: 487 ms, total: 3min 24s
Wall time: 3min 25s


In [164]:
%%time
CV_clf_sigmoid_rf_7 = calibrated_classifier(X_cap7_train, y_cap7_train, CV_clf_rf_7, 'calib_random_forest_classifier_7.sav')

#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_cal_random_camp7 = CV_clf_sigmoid_rf_7.predict_proba(X_cap7_test)[:, 1]
yhat_cal_random_camp7 = CV_clf_sigmoid_rf_7.predict(X_cap7_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap7_test, y_test_predict_cal_random_camp7, n_bins=10)

# Evaluate the accuracy
accuracy = accuracy_score(y_cap7_test, yhat_cal_random_camp7)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap7_test, yhat_cal_random_camp7))

calibrated_classifier started
calib_random_forest_classifier_7.sav  dumped !!!
Accuracy: 0.8690246659131706
              precision    recall  f1-score   support

           0       0.89      0.96      0.92     12100
           1       0.76      0.49      0.59      2941

    accuracy                           0.87     15041
   macro avg       0.82      0.72      0.76     15041
weighted avg       0.86      0.87      0.86     15041

CPU times: user 3min 34s, sys: 407 ms, total: 3min 34s
Wall time: 3min 36s


In [165]:
%%time
CV_clf_sigmoid_rf_8 = calibrated_classifier(X_cap8_train, y_cap8_train, CV_clf_rf_8, 'calib_random_forest_classifier_8.sav')

#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_cal_random_camp8 = CV_clf_sigmoid_rf_8.predict_proba(X_cap8_test)[:, 1]
yhat_cal_random_camp8 = CV_clf_sigmoid_rf_8.predict(X_cap8_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap8_test, y_test_predict_cal_random_camp8, n_bins=10)

# Evaluate the accuracy
accuracy = accuracy_score(y_cap8_test, yhat_cal_random_camp8)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap8_test, yhat_cal_random_camp8))

calibrated_classifier started
calib_random_forest_classifier_8.sav  dumped !!!
Accuracy: 0.8061890346451396
              precision    recall  f1-score   support

           0       0.81      0.99      0.89     11893
           1       0.61      0.09      0.15      2972

    accuracy                           0.81     14865
   macro avg       0.71      0.54      0.52     14865
weighted avg       0.77      0.81      0.74     14865

CPU times: user 3min 28s, sys: 530 ms, total: 3min 28s
Wall time: 3min 30s


In [166]:
%%time
CV_clf_sigmoid_rf_9 = calibrated_classifier(X_cap9_train, y_cap9_train, CV_clf_rf_9, 'calib_random_forest_classifier_9.sav')

#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_cal_random_camp9 = CV_clf_sigmoid_rf_9.predict_proba(X_cap9_test)[:, 1]
yhat_cal_random_camp9 = CV_clf_sigmoid_rf_9.predict(X_cap9_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap9_test, y_test_predict_cal_random_camp9, n_bins=10)

# Evaluate the accuracy
accuracy = accuracy_score(y_cap9_test, yhat_cal_random_camp9)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap9_test, yhat_cal_random_camp9))

calibrated_classifier started
calib_random_forest_classifier_9.sav  dumped !!!
Accuracy: 0.9932239420713479
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     12046
           1       0.97      1.00      0.98      3007

    accuracy                           0.99     15053
   macro avg       0.98      1.00      0.99     15053
weighted avg       0.99      0.99      0.99     15053

CPU times: user 2min 30s, sys: 321 ms, total: 2min 31s
Wall time: 2min 31s


In [167]:
%%time
CV_clf_sigmoid_rf_10 = calibrated_classifier(X_cap10_train, y_cap10_train, CV_clf_rf_10, 'calib_random_forest_classifier_10.sav')

#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_cal_random_camp10 = CV_clf_sigmoid_rf_10.predict_proba(X_cap10_test)[:, 1]
yhat_cal_random_camp10 = CV_clf_sigmoid_rf_10.predict(X_cap10_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap10_test, y_test_predict_cal_random_camp10, n_bins=10)

# Evaluate the accuracy
accuracy = accuracy_score(y_cap10_test, yhat_cal_random_camp10)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap10_test, yhat_cal_random_camp10))

calibrated_classifier started
calib_random_forest_classifier_10.sav  dumped !!!
Accuracy: 0.8660198046122151
              precision    recall  f1-score   support

           0       0.88      0.97      0.92     12045
           1       0.78      0.46      0.58      3002

    accuracy                           0.87     15047
   macro avg       0.83      0.71      0.75     15047
weighted avg       0.86      0.87      0.85     15047

CPU times: user 4min 8s, sys: 653 ms, total: 4min 8s
Wall time: 4min 54s


In [168]:
%%time
crf_results_camp1 = CV_clf_sigmoid_rf_1.predict_proba(X_prediction_camp1)[:, 1]
crf_results_camp2 = CV_clf_sigmoid_rf_2.predict_proba(X_prediction_camp2)[:, 1]
crf_results_camp3 = CV_clf_sigmoid_rf_3.predict_proba(X_prediction_camp3)[:, 1]
crf_results_camp4 = CV_clf_sigmoid_rf_4.predict_proba(X_prediction_camp4)[:, 1]
crf_results_camp5 = CV_clf_sigmoid_rf_5.predict_proba(X_prediction_camp5)[:, 1]
crf_results_camp6 = CV_clf_sigmoid_rf_6.predict_proba(X_prediction_camp6)[:, 1]
crf_results_camp7 = CV_clf_sigmoid_rf_7.predict_proba(X_prediction_camp7)[:, 1]
crf_results_camp8 = CV_clf_sigmoid_rf_8.predict_proba(X_prediction_camp8)[:, 1]
crf_results_camp9 = CV_clf_sigmoid_rf_9.predict_proba(X_prediction_camp9)[:, 1]
crf_results_camp10 = CV_clf_sigmoid_rf_10.predict_proba(X_prediction_camp10)[:, 1]

CPU times: user 4min 54s, sys: 446 ms, total: 4min 55s
Wall time: 5min 5s


In [169]:
final_results_rf['prob_results_crf_1'] = crf_results_camp1
final_results_rf['prob_results_crf_2'] = crf_results_camp2
final_results_rf['prob_results_crf_3'] = crf_results_camp3
final_results_rf['prob_results_crf_4'] = crf_results_camp4
final_results_rf['prob_results_crf_5'] = crf_results_camp5
final_results_rf['prob_results_crf_6'] = crf_results_camp6
final_results_rf['prob_results_rcf_7'] = crf_results_camp7
final_results_rf['prob_results_crf_8'] = crf_results_camp8
final_results_rf['prob_results_crf_9'] = crf_results_camp9
final_results_rf['prob_results_crf_10'] = crf_results_camp10
final_results_rf.head()

,SubscriberID,prob_results_rf_1,prob_results_rf_2,prob_results_rf_3,prob_results_rf_4,prob_results_rf_5,prob_results_rf_6,prob_results_rf_7,prob_results_rf_8,prob_results_rf_9,prob_results_rf_10,prob_results_crf_1,prob_results_crf_2,prob_results_crf_3,prob_results_crf_4,prob_results_crf_5,prob_results_crf_6,prob_results_rcf_7,prob_results_crf_8,prob_results_crf_9,prob_results_crf_10
1,4629739439,0.259914,0.217504,0.185521,0.238323,0.202069,0.067267,0.170452,0.190467,0.036856,0.163187,0.333959,0.233083,0.136440,0.242032,0.185094,0.009460,0.121855,0.171276,7.598090e-10,0.095296
2,10505531439,0.137936,0.251644,0.200538,0.315057,0.210328,0.056868,0.072705,0.213112,0.062082,0.093859,0.091364,0.399449,0.180556,0.478871,0.210394,0.008104,0.046636,0.224054,1.416328e-09,0.036748
3,12309121439,0.166742,0.192241,0.210715,0.120472,0.231834,0.070246,0.131997,0.189260,0.018200,0.068898,0.120844,0.135498,0.212009,0.060592,0.285501,0.009258,0.084712,0.167730,2.522110e-10,0.024995
4,13634651439,0.177301,0.215344,0.200381,0.168308,0.211474,0.100343,0.145285,0.191114,0.115662,0.179885,0.139033,0.227013,0.181250,0.116762,0.215963,0.014037,0.095123,0.172952,2.574307e-08,0.118829
5,13634651439,0.177235,0.215426,0.200238,0.184674,0.211443,0.104338,0.145285,0.191082,0.115662,0.175266,0.138937,0.226927,0.180427,0.144297,0.215973,0.015720,0.095163,0.172479,2.574307e-08,0.114183


In [171]:
final_results_rf.to_csv('rf_final_results_2.csv', index=False)

## Gaussian Naive Bias

In [179]:
def gaussian_naive_bias(x_train, y_train, filename1, filename2, filename3):
    print("gaussian_naive_bias_classifier started")

    # Create a pipeline with only Gaussian Naive Bayes
    pipeline = Pipeline([
        ('nb', GaussianNB())
    ])

    # Define the parameters you want to tune
    parameters = {
        'var_smoothing': [1e-9, 1e-8, 1e-7],  # Example parameter for Gaussian Naive Bayes
        'priors': [None, [0.2, 0.8], [0.5, 0.5]],  # Another parameter for Gaussian Naive Bayes
        'class_prior' : [0.3, 0.7]
        # Add more parameters as needed
    }
    # Create a GridSearchCV object for Uncalibrated
    #clf_nb = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)
    clf_nb = GaussianNB()

    # Fit the data to find the best parameters
    clf_nb.fit(x_train, y_train)

    #Dump model
    #'gaussian_naive_bias_classifier.sav'
    #Dump model
    # save the model to disk
    pickle.dump(clf_nb, open(filename1, 'wb'))
    print(filename1, ' dumped !!!')

    # Calibrated
    clf_sigmoid_nb = CalibratedClassifierCV(clf_nb, cv=10, method='isotonic')

    clf_sigmoid_nb.fit(x_train, y_train)

    #Dump model
    #'isotonic_calibrated_naive_bias_classifier.sav')
    #Dump model
    # save the model to disk
    pickle.dump(clf_sigmoid_nb, open(filename2, 'wb'))
    print(filename2, ' dumped !!!')

    # Calibrated, Platt
    clf_sigmoid_nb_calib_sig = CalibratedClassifierCV(clf_nb, cv=10, method='sigmoid')
    clf_sigmoid_nb_calib_sig.fit(x_train, y_train)

    #Dump model
    #'sigmoid_calibrated_naive_bias_classifier.sav')
    #Dump model
    # save the model to disk
    pickle.dump(clf_sigmoid_nb_calib_sig, open(filename3, 'wb'))
    print(filename3, ' dumped !!!')

    return clf_nb, clf_sigmoid_nb, clf_sigmoid_nb_calib_sig

In [180]:
%%time
clf_nb_1, clf_sigmoid_nb_1, clf_sigmoid_nb_calib_sig_1 = gaussian_naive_bias(X_cap1_train, y_cap1_train,
                                                                               'gaussian_naive_bias_classifier_1.sav',
                                                                               'isotonic_calibrated_naive_bias_classifier_1.sav',
                                                                               'sigmoid_calibrated_naive_bias_classifier_1.sav'
                                                                               )

gaussian_naive_bias_classifier started
gaussian_naive_bias_classifier_1.sav  dumped !!!
isotonic_calibrated_naive_bias_classifier_1.sav  dumped !!!
sigmoid_calibrated_naive_bias_classifier_1.sav  dumped !!!
CPU times: user 3.53 s, sys: 702 ms, total: 4.23 s
Wall time: 5 s


In [181]:
%%time
#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_nb_camp1 = clf_nb_1.predict_proba(X_cap1_test)[:, 1]
yhat_nb_camp1 = clf_nb_1.predict(X_cap1_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap1_test, y_test_predict_nb_camp1, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap1_test, yhat_nb_camp1)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap1_test, yhat_nb_camp1))
##############
#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_sigmoid_nb_camp1 = clf_sigmoid_nb_1.predict_proba(X_cap1_test)[:, 1]
yhat_sigmoid_nb_camp1 = clf_sigmoid_nb_1.predict(X_cap1_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap1_test, y_test_predict_sigmoid_nb_camp1, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap1_test, yhat_sigmoid_nb_camp1)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap1_test, yhat_sigmoid_nb_camp1))
##############
#X_cap1_train, X_cap1_test, y_cap1_train, y_cap1_test
y_test_predict_sigmoid_nb_calib_camp1 = clf_sigmoid_nb_calib_sig_1.predict_proba(X_cap1_test)[:, 1]
yhat_sigmoid_nb_calib_camp1 = clf_sigmoid_nb_calib_sig_1.predict(X_cap1_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap1_test, y_test_predict_sigmoid_nb_calib_camp1, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap1_test, yhat_sigmoid_nb_calib_camp1)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap1_test, yhat_sigmoid_nb_calib_camp1))

Accuracy: 0.20969917712127345
              precision    recall  f1-score   support

           0       0.90      0.02      0.03     11892
           1       0.20      0.99      0.33      2934

    accuracy                           0.21     14826
   macro avg       0.55      0.50      0.18     14826
weighted avg       0.76      0.21      0.09     14826

Accuracy: 0.802104411169567
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11892
           1       0.00      0.00      0.00      2934

    accuracy                           0.80     14826
   macro avg       0.40      0.50      0.45     14826
weighted avg       0.64      0.80      0.71     14826



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.802104411169567
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11892
           1       0.00      0.00      0.00      2934

    accuracy                           0.80     14826
   macro avg       0.40      0.50      0.45     14826
weighted avg       0.64      0.80      0.71     14826

CPU times: user 1.21 s, sys: 36.1 ms, total: 1.25 s
Wall time: 1.25 s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [183]:
%%time
clf_nb_2, clf_sigmoid_nb_2, clf_sigmoid_nb_calib_sig_2 = gaussian_naive_bias(X_cap2_train, y_cap2_train,
                                                                               'gaussian_naive_bias_classifier_2.sav',
                                                                               'isotonic_calibrated_naive_bias_classifier_2.sav',
                                                                               'sigmoid_calibrated_naive_bias_classifier_2.sav'
                                                                               )

#X_cap2_train, X_cap2_test, y_cap2_train, y_cap2_test
y_test_predict_nb_camp2 = clf_nb_2.predict_proba(X_cap2_test)[:, 1]
yhat_nb_camp2 = clf_nb_2.predict(X_cap2_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap2_test, y_test_predict_nb_camp2, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap2_test, yhat_nb_camp2)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap2_test, yhat_nb_camp2))
##############
#X_cap2_train, X_cap2_test, y_cap2_train, y_cap2_test
y_test_predict_sigmoid_nb_camp2 = clf_sigmoid_nb_2.predict_proba(X_cap2_test)[:, 1]
yhat_sigmoid_nb_camp2 = clf_sigmoid_nb_2.predict(X_cap2_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap2_test, y_test_predict_sigmoid_nb_camp2, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap2_test, yhat_sigmoid_nb_camp2)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap2_test, yhat_sigmoid_nb_camp2))
##############
#X_cap2_train, X_cap2_test, y_cap2_train, y_cap2_test
y_test_predict_sigmoid_nb_calib_camp2 = clf_sigmoid_nb_calib_sig_2.predict_proba(X_cap2_test)[:, 1]
yhat_sigmoid_nb_calib_camp2 = clf_sigmoid_nb_calib_sig_2.predict(X_cap2_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap2_test, y_test_predict_sigmoid_nb_calib_camp2, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap2_test, yhat_sigmoid_nb_calib_camp2)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap2_test, yhat_sigmoid_nb_calib_camp2))

gaussian_naive_bias_classifier started
gaussian_naive_bias_classifier_2.sav  dumped !!!
isotonic_calibrated_naive_bias_classifier_2.sav  dumped !!!
sigmoid_calibrated_naive_bias_classifier_2.sav  dumped !!!
Accuracy: 0.21986937091784117
              precision    recall  f1-score   support

           0       0.93      0.03      0.06     11666
           1       0.20      0.99      0.33      2879

    accuracy                           0.22     14545
   macro avg       0.57      0.51      0.20     14545
weighted avg       0.79      0.22      0.11     14545

Accuracy: 0.8020625644551392
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11666
           1       0.00      0.00      0.00      2879

    accuracy                           0.80     14545
   macro avg       0.40      0.50      0.45     14545
weighted avg       0.64      0.80      0.71     14545



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.8020625644551392
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11666
           1       0.00      0.00      0.00      2879

    accuracy                           0.80     14545
   macro avg       0.40      0.50      0.45     14545
weighted avg       0.64      0.80      0.71     14545

CPU times: user 4.07 s, sys: 16.6 ms, total: 4.09 s
Wall time: 4.53 s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [184]:
%%time
clf_nb_3, clf_sigmoid_nb_3, clf_sigmoid_nb_calib_sig_3 = gaussian_naive_bias(X_cap3_train, y_cap3_train,
                                                                               'gaussian_naive_bias_classifier_3.sav',
                                                                               'isotonic_calibrated_naive_bias_classifier_3.sav',
                                                                               'sigmoid_calibrated_naive_bias_classifier_3.sav'
                                                                               )

#X_cap3_train, X_cap3_test, y_cap3_train, y_cap3_test
y_test_predict_nb_camp3 = clf_nb_3.predict_proba(X_cap3_test)[:, 1]
yhat_nb_camp3 = clf_nb_3.predict(X_cap3_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap3_test, y_test_predict_nb_camp3, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap3_test, yhat_nb_camp3)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap3_test, yhat_nb_camp3))
##############
#X_cap3_train, X_cap3_test, y_cap3_train, y_cap3_test
y_test_predict_sigmoid_nb_camp3 = clf_sigmoid_nb_3.predict_proba(X_cap3_test)[:, 1]
yhat_sigmoid_nb_camp3 = clf_sigmoid_nb_3.predict(X_cap3_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap3_test, y_test_predict_sigmoid_nb_camp3, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap3_test, yhat_sigmoid_nb_camp3)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap3_test, yhat_sigmoid_nb_camp3))
##############
#X_cap3_train, X_cap3_test, y_cap3_train, y_cap3_test
y_test_predict_sigmoid_nb_calib_camp3 = clf_sigmoid_nb_calib_sig_3.predict_proba(X_cap3_test)[:, 1]
yhat_sigmoid_nb_calib_camp3 = clf_sigmoid_nb_calib_sig_3.predict(X_cap3_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap3_test, y_test_predict_sigmoid_nb_calib_camp3, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap3_test, yhat_sigmoid_nb_calib_camp3)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap3_test, yhat_sigmoid_nb_calib_camp3))

gaussian_naive_bias_classifier started
gaussian_naive_bias_classifier_3.sav  dumped !!!
isotonic_calibrated_naive_bias_classifier_3.sav  dumped !!!
sigmoid_calibrated_naive_bias_classifier_3.sav  dumped !!!
Accuracy: 0.21847620382701338
              precision    recall  f1-score   support

           0       0.95      0.03      0.05     11456
           1       0.20      0.99      0.33      2811

    accuracy                           0.22     14267
   macro avg       0.57      0.51      0.19     14267
weighted avg       0.80      0.22      0.11     14267

Accuracy: 0.8029718931800659
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11456
           1       0.00      0.00      0.00      2811

    accuracy                           0.80     14267
   macro avg       0.40      0.50      0.45     14267
weighted avg       0.64      0.80      0.72     14267



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.8029718931800659
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11456
           1       0.00      0.00      0.00      2811

    accuracy                           0.80     14267
   macro avg       0.40      0.50      0.45     14267
weighted avg       0.64      0.80      0.72     14267

CPU times: user 4.96 s, sys: 19 ms, total: 4.98 s
Wall time: 6.88 s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [185]:
%%time
clf_nb_4, clf_sigmoid_nb_4, clf_sigmoid_nb_calib_sig_4 = gaussian_naive_bias(X_cap4_train, y_cap4_train,
                                                                               'gaussian_naive_bias_classifier_4.sav',
                                                                               'isotonic_calibrated_naive_bias_classifier_4.sav',
                                                                               'sigmoid_calibrated_naive_bias_classifier_4.sav'
                                                                               )

#X_cap4_train, X_cap4_test, y_cap4_train, y_cap4_test
y_test_predict_nb_camp4 = clf_nb_4.predict_proba(X_cap4_test)[:, 1]
yhat_nb_camp4 = clf_nb_4.predict(X_cap4_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap4_test, y_test_predict_nb_camp4, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap4_test, yhat_nb_camp4)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap4_test, yhat_nb_camp4))
##############
#X_cap4_train, X_cap4_test, y_cap4_train, y_cap4_test
y_test_predict_sigmoid_nb_camp4 = clf_sigmoid_nb_4.predict_proba(X_cap4_test)[:, 1]
yhat_sigmoid_nb_camp4 = clf_sigmoid_nb_4.predict(X_cap4_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap4_test, y_test_predict_sigmoid_nb_camp4, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap4_test, yhat_sigmoid_nb_camp4)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap4_test, yhat_sigmoid_nb_camp4))
##############
#X_cap4_train, X_cap4_test, y_cap4_train, y_cap4_test
y_test_predict_sigmoid_nb_calib_camp4 = clf_sigmoid_nb_calib_sig_4.predict_proba(X_cap4_test)[:, 1]
yhat_sigmoid_nb_calib_camp4 = clf_sigmoid_nb_calib_sig_4.predict(X_cap4_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap4_test, y_test_predict_sigmoid_nb_calib_camp4, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap4_test, yhat_sigmoid_nb_calib_camp4)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap4_test, yhat_sigmoid_nb_calib_camp4))

gaussian_naive_bias_classifier started
gaussian_naive_bias_classifier_4.sav  dumped !!!
isotonic_calibrated_naive_bias_classifier_4.sav  dumped !!!
sigmoid_calibrated_naive_bias_classifier_4.sav  dumped !!!
Accuracy: 0.2903462050599201
              precision    recall  f1-score   support

           0       1.00      0.12      0.21     12069
           1       0.22      1.00      0.36      2951

    accuracy                           0.29     15020
   macro avg       0.61      0.56      0.28     15020
weighted avg       0.85      0.29      0.24     15020

Accuracy: 0.8035286284953396
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     12069
           1       0.00      0.00      0.00      2951

    accuracy                           0.80     15020
   macro avg       0.40      0.50      0.45     15020
weighted avg       0.65      0.80      0.72     15020



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.8035286284953396
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     12069
           1       0.00      0.00      0.00      2951

    accuracy                           0.80     15020
   macro avg       0.40      0.50      0.45     15020
weighted avg       0.65      0.80      0.72     15020

CPU times: user 4.61 s, sys: 18.4 ms, total: 4.63 s
Wall time: 4.72 s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [186]:
%%time
clf_nb_5, clf_sigmoid_nb_5, clf_sigmoid_nb_calib_sig_5 = gaussian_naive_bias(X_cap5_train, y_cap5_train,
                                                                               'gaussian_naive_bias_classifier_5.sav',
                                                                               'isotonic_calibrated_naive_bias_classifier_5.sav',
                                                                               'sigmoid_calibrated_naive_bias_classifier_5.sav'
                                                                               )

#X_cap5_train, X_cap5_test, y_cap5_train, y_cap5_test
y_test_predict_nb_camp5 = clf_nb_5.predict_proba(X_cap5_test)[:, 1]
yhat_nb_camp5 = clf_nb_5.predict(X_cap5_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap5_test, y_test_predict_nb_camp5, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap5_test, yhat_nb_camp5)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap5_test, yhat_nb_camp5))
##############
#X_cap5_train, X_cap5_test, y_cap5_train, y_cap5_test
y_test_predict_sigmoid_nb_camp5 = clf_sigmoid_nb_5.predict_proba(X_cap5_test)[:, 1]
yhat_sigmoid_nb_camp5 = clf_sigmoid_nb_5.predict(X_cap5_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap5_test, y_test_predict_sigmoid_nb_camp5, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap5_test, yhat_sigmoid_nb_camp5)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap5_test, yhat_sigmoid_nb_camp5))
##############
#X_cap5_train, X_cap5_test, y_cap5_train, y_cap5_test
y_test_predict_sigmoid_nb_calib_camp5 = clf_sigmoid_nb_calib_sig_5.predict_proba(X_cap5_test)[:, 1]
yhat_sigmoid_nb_calib_camp5 = clf_sigmoid_nb_calib_sig_5.predict(X_cap5_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap5_test, y_test_predict_sigmoid_nb_calib_camp5, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap5_test, yhat_sigmoid_nb_calib_camp5)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap5_test, yhat_sigmoid_nb_calib_camp5))

gaussian_naive_bias_classifier started
gaussian_naive_bias_classifier_5.sav  dumped !!!
isotonic_calibrated_naive_bias_classifier_5.sav  dumped !!!
sigmoid_calibrated_naive_bias_classifier_5.sav  dumped !!!
Accuracy: 0.21427085500312046
              precision    recall  f1-score   support

           0       0.92      0.02      0.05     11580
           1       0.20      0.99      0.33      2841

    accuracy                           0.21     14421
   macro avg       0.56      0.51      0.19     14421
weighted avg       0.78      0.21      0.10     14421

Accuracy: 0.8029956313709175
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11580
           1       0.00      0.00      0.00      2841

    accuracy                           0.80     14421
   macro avg       0.40      0.50      0.45     14421
weighted avg       0.64      0.80      0.72     14421



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.8029956313709175
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11580
           1       0.00      0.00      0.00      2841

    accuracy                           0.80     14421
   macro avg       0.40      0.50      0.45     14421
weighted avg       0.64      0.80      0.72     14421

CPU times: user 3.94 s, sys: 8.43 ms, total: 3.95 s
Wall time: 4.06 s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [187]:
%%time
clf_nb_6, clf_sigmoid_nb_6, clf_sigmoid_nb_calib_sig_6 = gaussian_naive_bias(X_cap6_train, y_cap6_train,
                                                                               'gaussian_naive_bias_classifier_6.sav',
                                                                               'isotonic_calibrated_naive_bias_classifier_6.sav',
                                                                               'sigmoid_calibrated_naive_bias_classifier_6.sav'
                                                                               )

#X_cap6_train, X_cap6_test, y_cap6_train, y_cap6_test
y_test_predict_nb_camp6 = clf_nb_6.predict_proba(X_cap6_test)[:, 1]
yhat_nb_camp6 = clf_nb_6.predict(X_cap6_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap6_test, y_test_predict_nb_camp6, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap6_test, yhat_nb_camp6)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap6_test, yhat_nb_camp6))
##############
#X_cap6_train, X_cap6_test, y_cap6_train, y_cap6_test
y_test_predict_sigmoid_nb_camp6 = clf_sigmoid_nb_6.predict_proba(X_cap6_test)[:, 1]
yhat_sigmoid_nb_camp6 = clf_sigmoid_nb_6.predict(X_cap6_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap6_test, y_test_predict_sigmoid_nb_camp6, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap6_test, yhat_sigmoid_nb_camp6)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap6_test, yhat_sigmoid_nb_camp6))
##############
#X_cap6_train, X_cap6_test, y_cap6_train, y_cap6_test
y_test_predict_sigmoid_nb_calib_camp6 = clf_sigmoid_nb_calib_sig_6.predict_proba(X_cap6_test)[:, 1]
yhat_sigmoid_nb_calib_camp6 = clf_sigmoid_nb_calib_sig_6.predict(X_cap6_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap6_test, y_test_predict_sigmoid_nb_calib_camp6, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap6_test, yhat_sigmoid_nb_calib_camp6)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap6_test, yhat_sigmoid_nb_calib_camp6))

gaussian_naive_bias_classifier started
gaussian_naive_bias_classifier_6.sav  dumped !!!
isotonic_calibrated_naive_bias_classifier_6.sav  dumped !!!
sigmoid_calibrated_naive_bias_classifier_6.sav  dumped !!!
Accuracy: 0.42070294332602487
              precision    recall  f1-score   support

           0       1.00      0.28      0.44     12079
           1       0.25      1.00      0.41      2972

    accuracy                           0.42     15051
   macro avg       0.63      0.64      0.42     15051
weighted avg       0.85      0.42      0.43     15051

Accuracy: 0.8025380373397116
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     12079
           1       0.00      0.00      0.00      2972

    accuracy                           0.80     15051
   macro avg       0.40      0.50      0.45     15051
weighted avg       0.64      0.80      0.71     15051



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.8025380373397116
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     12079
           1       0.00      0.00      0.00      2972

    accuracy                           0.80     15051
   macro avg       0.40      0.50      0.45     15051
weighted avg       0.64      0.80      0.71     15051

CPU times: user 3.72 s, sys: 518 ms, total: 4.24 s
Wall time: 4.23 s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [188]:
%%time
clf_nb_7, clf_sigmoid_nb_7, clf_sigmoid_nb_calib_sig_7 = gaussian_naive_bias(X_cap7_train, y_cap7_train,
                                                                               'gaussian_naive_bias_classifier_7.sav',
                                                                               'isotonic_calibrated_naive_bias_classifier_7.sav',
                                                                               'sigmoid_calibrated_naive_bias_classifier_7.sav'
                                                                               )

#X_cap7_train, X_cap7_test, y_cap7_train, y_cap7_test
y_test_predict_nb_camp7 = clf_nb_7.predict_proba(X_cap7_test)[:, 1]
yhat_nb_camp7 = clf_nb_7.predict(X_cap7_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap7_test, y_test_predict_nb_camp7, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap7_test, yhat_nb_camp7)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap7_test, yhat_nb_camp7))
##############
#X_cap7_train, X_cap7_test, y_cap7_train, y_cap7_test
y_test_predict_sigmoid_nb_camp7 = clf_sigmoid_nb_7.predict_proba(X_cap7_test)[:, 1]
yhat_sigmoid_nb_camp7 = clf_sigmoid_nb_7.predict(X_cap7_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap7_test, y_test_predict_sigmoid_nb_camp7, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap7_test, yhat_sigmoid_nb_camp7)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap7_test, yhat_sigmoid_nb_camp7))
##############
#X_cap7_train, X_cap7_test, y_cap7_train, y_cap7_test
y_test_predict_sigmoid_nb_calib_camp7 = clf_sigmoid_nb_calib_sig_7.predict_proba(X_cap7_test)[:, 1]
yhat_sigmoid_nb_calib_camp7 = clf_sigmoid_nb_calib_sig_7.predict(X_cap7_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap7_test, y_test_predict_sigmoid_nb_calib_camp7, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap7_test, yhat_sigmoid_nb_calib_camp7)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap7_test, yhat_sigmoid_nb_calib_camp7))

gaussian_naive_bias_classifier started
gaussian_naive_bias_classifier_7.sav  dumped !!!
isotonic_calibrated_naive_bias_classifier_7.sav  dumped !!!
sigmoid_calibrated_naive_bias_classifier_7.sav  dumped !!!
Accuracy: 0.30443454557542715
              precision    recall  f1-score   support

           0       1.00      0.14      0.24     12100
           1       0.22      1.00      0.36      2941

    accuracy                           0.30     15041
   macro avg       0.61      0.57      0.30     15041
weighted avg       0.85      0.30      0.26     15041

Accuracy: 0.8044677880460076
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     12100
           1       0.00      0.00      0.00      2941

    accuracy                           0.80     15041
   macro avg       0.40      0.50      0.45     15041
weighted avg       0.65      0.80      0.72     15041



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.8044677880460076
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     12100
           1       0.00      0.00      0.00      2941

    accuracy                           0.80     15041
   macro avg       0.40      0.50      0.45     15041
weighted avg       0.65      0.80      0.72     15041

CPU times: user 3.47 s, sys: 25.4 ms, total: 3.5 s
Wall time: 3.79 s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [189]:
%%time
clf_nb_8, clf_sigmoid_nb_8, clf_sigmoid_nb_calib_sig_8 = gaussian_naive_bias(X_cap8_train, y_cap8_train,
                                                                               'gaussian_naive_bias_classifier_8.sav',
                                                                               'isotonic_calibrated_naive_bias_classifier_8.sav',
                                                                               'sigmoid_calibrated_naive_bias_classifier_8.sav'
                                                                               )

#X_cap8_train, X_cap8_test, y_cap8_train, y_cap8_test
y_test_predict_nb_camp8 = clf_nb_8.predict_proba(X_cap8_test)[:, 1]
yhat_nb_camp8 = clf_nb_8.predict(X_cap8_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap8_test, y_test_predict_nb_camp8, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap8_test, yhat_nb_camp8)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap8_test, yhat_nb_camp8))
##############
#X_cap8_train, X_cap8_test, y_cap8_train, y_cap8_test
y_test_predict_sigmoid_nb_camp8 = clf_sigmoid_nb_8.predict_proba(X_cap8_test)[:, 1]
yhat_sigmoid_nb_camp8 = clf_sigmoid_nb_8.predict(X_cap8_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap8_test, y_test_predict_sigmoid_nb_camp8, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap8_test, yhat_sigmoid_nb_camp8)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap8_test, yhat_sigmoid_nb_camp8))
##############
#X_cap8_train, X_cap8_test, y_cap8_train, y_cap8_test
y_test_predict_sigmoid_nb_calib_camp8 = clf_sigmoid_nb_calib_sig_8.predict_proba(X_cap8_test)[:, 1]
yhat_sigmoid_nb_calib_camp8 = clf_sigmoid_nb_calib_sig_8.predict(X_cap8_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap8_test, y_test_predict_sigmoid_nb_calib_camp8, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap8_test, yhat_sigmoid_nb_calib_camp8)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap8_test, yhat_sigmoid_nb_calib_camp8))

gaussian_naive_bias_classifier started
gaussian_naive_bias_classifier_8.sav  dumped !!!
isotonic_calibrated_naive_bias_classifier_8.sav  dumped !!!
sigmoid_calibrated_naive_bias_classifier_8.sav  dumped !!!
Accuracy: 0.217221661621258
              precision    recall  f1-score   support

           0       0.97      0.02      0.04     11893
           1       0.20      1.00      0.34      2972

    accuracy                           0.22     14865
   macro avg       0.59      0.51      0.19     14865
weighted avg       0.82      0.22      0.10     14865

Accuracy: 0.800067272115708
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11893
           1       0.00      0.00      0.00      2972

    accuracy                           0.80     14865
   macro avg       0.40      0.50      0.44     14865
weighted avg       0.64      0.80      0.71     14865



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.800067272115708
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     11893
           1       0.00      0.00      0.00      2972

    accuracy                           0.80     14865
   macro avg       0.40      0.50      0.44     14865
weighted avg       0.64      0.80      0.71     14865

CPU times: user 5.22 s, sys: 15.2 ms, total: 5.24 s
Wall time: 6.77 s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [190]:
%%time
clf_nb_9, clf_sigmoid_nb_9, clf_sigmoid_nb_calib_sig_9 = gaussian_naive_bias(X_cap9_train, y_cap9_train,
                                                                               'gaussian_naive_bias_classifier_9.sav',
                                                                               'isotonic_calibrated_naive_bias_classifier_9.sav',
                                                                               'sigmoid_calibrated_naive_bias_classifier_9.sav'
                                                                               )

#X_cap9_train, X_cap9_test, y_cap9_train, y_cap9_test
y_test_predict_nb_camp9 = clf_nb_9.predict_proba(X_cap9_test)[:, 1]
yhat_nb_camp9 = clf_nb_9.predict(X_cap9_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap9_test, y_test_predict_nb_camp9, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap9_test, yhat_nb_camp9)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap9_test, yhat_nb_camp9))
##############
#X_cap9_train, X_cap9_test, y_cap9_train, y_cap9_test
y_test_predict_sigmoid_nb_camp9 = clf_sigmoid_nb_9.predict_proba(X_cap9_test)[:, 1]
yhat_sigmoid_nb_camp9 = clf_sigmoid_nb_9.predict(X_cap9_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap9_test, y_test_predict_sigmoid_nb_camp9, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap9_test, yhat_sigmoid_nb_camp9)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap9_test, yhat_sigmoid_nb_camp9))
##############
#X_cap9_train, X_cap9_test, y_cap9_train, y_cap9_test
y_test_predict_sigmoid_nb_calib_camp9 = clf_sigmoid_nb_calib_sig_9.predict_proba(X_cap9_test)[:, 1]
yhat_sigmoid_nb_calib_camp9 = clf_sigmoid_nb_calib_sig_9.predict(X_cap9_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap9_test, y_test_predict_sigmoid_nb_calib_camp9, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap9_test, yhat_sigmoid_nb_calib_camp9)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap9_test, yhat_sigmoid_nb_calib_camp9))

gaussian_naive_bias_classifier started
gaussian_naive_bias_classifier_9.sav  dumped !!!
isotonic_calibrated_naive_bias_classifier_9.sav  dumped !!!
sigmoid_calibrated_naive_bias_classifier_9.sav  dumped !!!
Accuracy: 0.9934232378927789
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     12046
           1       0.97      1.00      0.98      3007

    accuracy                           0.99     15053
   macro avg       0.98      1.00      0.99     15053
weighted avg       0.99      0.99      0.99     15053

Accuracy: 0.9934232378927789
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     12046
           1       0.97      1.00      0.98      3007

    accuracy                           0.99     15053
   macro avg       0.98      1.00      0.99     15053
weighted avg       0.99      0.99      0.99     15053

Accuracy: 0.9934232378927789
              precision    recall  f1-score   suppor

In [191]:
%%time
clf_nb_10, clf_sigmoid_nb_10, clf_sigmoid_nb_calib_sig_10 = gaussian_naive_bias(X_cap10_train, y_cap10_train,
                                                                               'gaussian_naive_bias_classifier_10.sav',
                                                                               'isotonic_calibrated_naive_bias_classifier_10.sav',
                                                                               'sigmoid_calibrated_naive_bias_classifier_10.sav'
                                                                               )

#X_cap10_train, X_cap10_test, y_cap10_train, y_cap10_test
y_test_predict_nb_camp10 = clf_nb_10.predict_proba(X_cap10_test)[:, 1]
yhat_nb_camp10 = clf_nb_10.predict(X_cap10_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap10_test, y_test_predict_nb_camp10, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap10_test, yhat_nb_camp10)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap10_test, yhat_nb_camp10))
##############
#X_cap10_train, X_cap10_test, y_cap10_train, y_cap10_test
y_test_predict_sigmoid_nb_camp10 = clf_sigmoid_nb_10.predict_proba(X_cap10_test)[:, 1]
yhat_sigmoid_nb_camp10 = clf_sigmoid_nb_10.predict(X_cap10_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap10_test, y_test_predict_sigmoid_nb_camp10, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap10_test, yhat_sigmoid_nb_camp10)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap10_test, yhat_sigmoid_nb_camp10))
##############
#X_cap10_train, X_cap10_test, y_cap10_train, y_cap10_test
y_test_predict_sigmoid_nb_calib_camp10 = clf_sigmoid_nb_calib_sig_10.predict_proba(X_cap10_test)[:, 1]
yhat_sigmoid_nb_calib_camp10 = clf_sigmoid_nb_calib_sig_10.predict(X_cap10_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_cap10_test, y_test_predict_sigmoid_nb_calib_camp10, n_bins=10)


# Evaluate the accuracy
accuracy = accuracy_score(y_cap10_test, yhat_sigmoid_nb_calib_camp10)
print(f"Accuracy: {accuracy}")

print(classification_report(y_cap10_test, yhat_sigmoid_nb_calib_camp10))

gaussian_naive_bias_classifier started
gaussian_naive_bias_classifier_10.sav  dumped !!!
isotonic_calibrated_naive_bias_classifier_10.sav  dumped !!!
sigmoid_calibrated_naive_bias_classifier_10.sav  dumped !!!
Accuracy: 0.33661194922575927
              precision    recall  f1-score   support

           0       1.00      0.17      0.29     12045
           1       0.23      1.00      0.38      3002

    accuracy                           0.34     15047
   macro avg       0.62      0.59      0.33     15047
weighted avg       0.85      0.34      0.31     15047

Accuracy: 0.8004917923838639
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     12045
           1       0.00      0.00      0.00      3002

    accuracy                           0.80     15047
   macro avg       0.40      0.50      0.44     15047
weighted avg       0.64      0.80      0.71     15047



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.8004917923838639
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     12045
           1       0.00      0.00      0.00      3002

    accuracy                           0.80     15047
   macro avg       0.40      0.50      0.44     15047
weighted avg       0.64      0.80      0.71     15047

CPU times: user 5.14 s, sys: 719 ms, total: 5.86 s
Wall time: 8 s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [192]:
%%time
nb_results_camp1 = clf_nb_1.predict_proba(X_prediction_camp1)[:, 1]
nb_results_camp2 = clf_nb_2.predict_proba(X_prediction_camp2)[:, 1]
nb_results_camp3 = clf_nb_3.predict_proba(X_prediction_camp3)[:, 1]
nb_results_camp4 = clf_nb_4.predict_proba(X_prediction_camp4)[:, 1]
nb_results_camp5 = clf_nb_5.predict_proba(X_prediction_camp5)[:, 1]
nb_results_camp6 = clf_nb_6.predict_proba(X_prediction_camp6)[:, 1]
nb_results_camp7 = clf_nb_7.predict_proba(X_prediction_camp7)[:, 1]
nb_results_camp8 = clf_nb_8.predict_proba(X_prediction_camp8)[:, 1]
nb_results_camp9 = clf_nb_9.predict_proba(X_prediction_camp9)[:, 1]
nb_results_camp10 = clf_nb_10.predict_proba(X_prediction_camp10)[:, 1]

CPU times: user 880 ms, sys: 544 ms, total: 1.42 s
Wall time: 1.42 s


In [193]:
final_results_rf['prob_results_nb_1'] = nb_results_camp1
final_results_rf['prob_results_nb_2'] = nb_results_camp2
final_results_rf['prob_results_nb_3'] = nb_results_camp3
final_results_rf['prob_results_nb_4'] = nb_results_camp4
final_results_rf['prob_results_nb_5'] = nb_results_camp5
final_results_rf['prob_results_nb_6'] = nb_results_camp6
final_results_rf['prob_results_nb_7'] = nb_results_camp7
final_results_rf['prob_results_nb_8'] = nb_results_camp8
final_results_rf['prob_results_nb_9'] = nb_results_camp9
final_results_rf['prob_results_nb_10'] = nb_results_camp10
final_results_rf.head()

,SubscriberID,prob_results_rf_1,prob_results_rf_2,prob_results_rf_3,prob_results_rf_4,prob_results_rf_5,prob_results_rf_6,prob_results_rf_7,prob_results_rf_8,prob_results_rf_9,prob_results_rf_10,prob_results_crf_1,prob_results_crf_2,prob_results_crf_3,prob_results_crf_4,prob_results_crf_5,prob_results_crf_6,prob_results_rcf_7,prob_results_crf_8,prob_results_crf_9,prob_results_crf_10,prob_results_nb_1,prob_results_nb_2,prob_results_nb_3,prob_results_nb_4,prob_results_nb_5,prob_results_nb_6,prob_results_nb_7,prob_results_nb_8,prob_results_nb_9,prob_results_nb_10
1,4629739439,0.259914,0.217504,0.185521,0.238323,0.202069,0.067267,0.170452,0.190467,0.036856,0.163187,0.333959,0.233083,0.136440,0.242032,0.185094,0.009460,0.121855,0.171276,7.598090e-10,0.095296,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0
2,10505531439,0.137936,0.251644,0.200538,0.315057,0.210328,0.056868,0.072705,0.213112,0.062082,0.093859,0.091364,0.399449,0.180556,0.478871,0.210394,0.008104,0.046636,0.224054,1.416328e-09,0.036748,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
3,12309121439,0.166742,0.192241,0.210715,0.120472,0.231834,0.070246,0.131997,0.189260,0.018200,0.068898,0.120844,0.135498,0.212009,0.060592,0.285501,0.009258,0.084712,0.167730,2.522110e-10,0.024995,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
4,13634651439,0.177301,0.215344,0.200381,0.168308,0.211474,0.100343,0.145285,0.191114,0.115662,0.179885,0.139033,0.227013,0.181250,0.116762,0.215963,0.014037,0.095123,0.172952,2.574307e-08,0.118829,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
5,13634651439,0.177235,0.215426,0.200238,0.184674,0.211443,0.104338,0.145285,0.191082,0.115662,0.175266,0.138937,0.226927,0.180427,0.144297,0.215973,0.015720,0.095163,0.172479,2.574307e-08,0.114183,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0


In [194]:
final_results_rf.shape

(60213, 31)

In [195]:
%%time
sigmoid_nb_results_camp1 = clf_sigmoid_nb_1.predict_proba(X_prediction_camp1)[:, 1]
sigmoid_nb_results_camp2 = clf_sigmoid_nb_2.predict_proba(X_prediction_camp2)[:, 1]
sigmoid_nb_results_camp3 = clf_sigmoid_nb_3.predict_proba(X_prediction_camp3)[:, 1]
sigmoid_nb_results_camp4 = clf_sigmoid_nb_4.predict_proba(X_prediction_camp4)[:, 1]
sigmoid_nb_results_camp5 = clf_sigmoid_nb_5.predict_proba(X_prediction_camp5)[:, 1]
sigmoid_nb_results_camp6 = clf_sigmoid_nb_6.predict_proba(X_prediction_camp6)[:, 1]
sigmoid_nb_results_camp7 = clf_sigmoid_nb_7.predict_proba(X_prediction_camp7)[:, 1]
sigmoid_nb_results_camp8 = clf_sigmoid_nb_8.predict_proba(X_prediction_camp8)[:, 1]
sigmoid_nb_results_camp9 = clf_sigmoid_nb_9.predict_proba(X_prediction_camp9)[:, 1]
sigmoid_nb_results_camp10 = clf_sigmoid_nb_10.predict_proba(X_prediction_camp10)[:, 1]

final_results_rf['prob_results_sigmoid_nb_1'] = sigmoid_nb_results_camp1
final_results_rf['prob_results_sigmoid_nb_2'] = sigmoid_nb_results_camp2
final_results_rf['prob_results_sigmoid_nb_3'] = sigmoid_nb_results_camp3
final_results_rf['prob_results_sigmoid_nb_4'] = sigmoid_nb_results_camp4
final_results_rf['prob_results_sigmoid_nb_5'] = sigmoid_nb_results_camp5
final_results_rf['prob_results_sigmoid_nb_6'] = sigmoid_nb_results_camp6
final_results_rf['prob_results_sigmoid_nb_7'] = sigmoid_nb_results_camp7
final_results_rf['prob_results_sigmoid_nb_8'] = sigmoid_nb_results_camp8
final_results_rf['prob_results_sigmoid_nb_9'] = sigmoid_nb_results_camp9
final_results_rf['prob_results_sigmoid_nb_10'] = sigmoid_nb_results_camp10
final_results_rf.head()

CPU times: user 10 s, sys: 6.87 s, total: 16.9 s
Wall time: 19.2 s


,SubscriberID,prob_results_rf_1,prob_results_rf_2,prob_results_rf_3,prob_results_rf_4,prob_results_rf_5,prob_results_rf_6,prob_results_rf_7,prob_results_rf_8,prob_results_rf_9,prob_results_rf_10,prob_results_crf_1,prob_results_crf_2,prob_results_crf_3,prob_results_crf_4,prob_results_crf_5,prob_results_crf_6,prob_results_rcf_7,prob_results_crf_8,prob_results_crf_9,prob_results_crf_10,prob_results_nb_1,prob_results_nb_2,prob_results_nb_3,prob_results_nb_4,prob_results_nb_5,prob_results_nb_6,prob_results_nb_7,prob_results_nb_8,prob_results_nb_9,prob_results_nb_10,prob_results_sigmoid_nb_1,prob_results_sigmoid_nb_2,prob_results_sigmoid_nb_3,prob_results_sigmoid_nb_4,prob_results_sigmoid_nb_5,prob_results_sigmoid_nb_6,prob_results_sigmoid_nb_7,prob_results_sigmoid_nb_8,prob_results_sigmoid_nb_9,prob_results_sigmoid_nb_10
1,4629739439,0.259914,0.217504,0.185521,0.238323,0.202069,0.067267,0.170452,0.190467,0.036856,0.163187,0.333959,0.233083,0.136440,0.242032,0.185094,0.009460,0.121855,0.171276,7.598090e-10,0.095296,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.202976,0.205429,0.204756,0.221636,0.203624,0.000000,0.225374,0.203749,0.0,0.229609
2,10505531439,0.137936,0.251644,0.200538,0.315057,0.210328,0.056868,0.072705,0.213112,0.062082,0.093859,0.091364,0.399449,0.180556,0.478871,0.210394,0.008104,0.046636,0.224054,1.416328e-09,0.036748,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.202976,0.205429,0.204756,0.221636,0.203624,0.000000,0.000000,0.203749,0.0,0.000000
3,12309121439,0.166742,0.192241,0.210715,0.120472,0.231834,0.070246,0.131997,0.189260,0.018200,0.068898,0.120844,0.135498,0.212009,0.060592,0.285501,0.009258,0.084712,0.167730,2.522110e-10,0.024995,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.202976,0.205429,0.204756,0.221636,0.203624,0.257463,0.225374,0.203749,0.0,0.229609
4,13634651439,0.177301,0.215344,0.200381,0.168308,0.211474,0.100343,0.145285,0.191114,0.115662,0.179885,0.139033,0.227013,0.181250,0.116762,0.215963,0.014037,0.095123,0.172952,2.574307e-08,0.118829,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.202976,0.205429,0.204756,0.221636,0.203624,0.257463,0.225374,0.203749,0.0,0.229609
5,13634651439,0.177235,0.215426,0.200238,0.184674,0.211443,0.104338,0.145285,0.191082,0.115662,0.175266,0.138937,0.226927,0.180427,0.144297,0.215973,0.015720,0.095163,0.172479,2.574307e-08,0.114183,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.202976,0.205429,0.204756,0.221636,0.203624,0.257463,0.225374,0.203749,0.0,0.229609


In [196]:
final_results_rf.shape

(60213, 41)

In [197]:
%%time
cal_sigmoid_nb_results_camp1 = clf_sigmoid_nb_calib_sig_1.predict_proba(X_prediction_camp1)[:, 1]
cal_sigmoid_nb_results_camp2 = clf_sigmoid_nb_calib_sig_2.predict_proba(X_prediction_camp2)[:, 1]
cal_sigmoid_nb_results_camp3 = clf_sigmoid_nb_calib_sig_3.predict_proba(X_prediction_camp3)[:, 1]
cal_sigmoid_nb_results_camp4 = clf_sigmoid_nb_calib_sig_4.predict_proba(X_prediction_camp4)[:, 1]
cal_sigmoid_nb_results_camp5 = clf_sigmoid_nb_calib_sig_5.predict_proba(X_prediction_camp5)[:, 1]
cal_sigmoid_nb_results_camp6 = clf_sigmoid_nb_calib_sig_6.predict_proba(X_prediction_camp6)[:, 1]
cal_sigmoid_nb_results_camp7 = clf_sigmoid_nb_calib_sig_7.predict_proba(X_prediction_camp7)[:, 1]
cal_sigmoid_nb_results_camp8 = clf_sigmoid_nb_calib_sig_8.predict_proba(X_prediction_camp8)[:, 1]
cal_sigmoid_nb_results_camp9 = clf_sigmoid_nb_calib_sig_9.predict_proba(X_prediction_camp9)[:, 1]
cal_sigmoid_nb_results_camp10 = clf_sigmoid_nb_calib_sig_10.predict_proba(X_prediction_camp10)[:, 1]

final_results_rf['prob_results_cal_sigmoid_nb_1'] = cal_sigmoid_nb_results_camp1
final_results_rf['prob_results_cal_sigmoid_nb_2'] = cal_sigmoid_nb_results_camp2
final_results_rf['prob_results_cal_sigmoid_nb_3'] = cal_sigmoid_nb_results_camp3
final_results_rf['prob_results_cal_sigmoid_nb_4'] = cal_sigmoid_nb_results_camp4
final_results_rf['prob_results_cal_sigmoid_nb_5'] = cal_sigmoid_nb_results_camp5
final_results_rf['prob_results_cal_sigmoid_nb_6'] = cal_sigmoid_nb_results_camp6
final_results_rf['prob_results_cal_sigmoid_nb_7'] = cal_sigmoid_nb_results_camp7
final_results_rf['prob_results_cal_sigmoid_nb_8'] = cal_sigmoid_nb_results_camp8
final_results_rf['prob_results_cal_sigmoid_nb_9'] = cal_sigmoid_nb_results_camp9
final_results_rf['prob_results_cal_sigmoid_nb_10'] = cal_sigmoid_nb_results_camp10
final_results_rf.head()

CPU times: user 9.2 s, sys: 6.13 s, total: 15.3 s
Wall time: 17 s


,SubscriberID,prob_results_rf_1,prob_results_rf_2,prob_results_rf_3,prob_results_rf_4,prob_results_rf_5,prob_results_rf_6,prob_results_rf_7,prob_results_rf_8,prob_results_rf_9,prob_results_rf_10,prob_results_crf_1,prob_results_crf_2,prob_results_crf_3,prob_results_crf_4,prob_results_crf_5,prob_results_crf_6,prob_results_rcf_7,prob_results_crf_8,prob_results_crf_9,prob_results_crf_10,prob_results_nb_1,prob_results_nb_2,prob_results_nb_3,prob_results_nb_4,prob_results_nb_5,prob_results_nb_6,prob_results_nb_7,prob_results_nb_8,prob_results_nb_9,prob_results_nb_10,prob_results_sigmoid_nb_1,prob_results_sigmoid_nb_2,prob_results_sigmoid_nb_3,prob_results_sigmoid_nb_4,prob_results_sigmoid_nb_5,prob_results_sigmoid_nb_6,prob_results_sigmoid_nb_7,prob_results_sigmoid_nb_8,prob_results_sigmoid_nb_9,prob_results_sigmoid_nb_10,prob_results_cal_sigmoid_nb_1,prob_results_cal_sigmoid_nb_2,prob_results_cal_sigmoid_nb_3,prob_results_cal_sigmoid_nb_4,prob_results_cal_sigmoid_nb_5,prob_results_cal_sigmoid_nb_6,prob_results_cal_sigmoid_nb_7,prob_results_cal_sigmoid_nb_8,prob_results_cal_sigmoid_nb_9,prob_results_cal_sigmoid_nb_10
1,4629739439,0.259914,0.217504,0.185521,0.238323,0.202069,0.067267,0.170452,0.190467,0.036856,0.163187,0.333959,0.233083,0.136440,0.242032,0.185094,0.009460,0.121855,0.171276,7.598090e-10,0.095296,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.202976,0.205429,0.204756,0.221636,0.203624,0.000000,0.225374,0.203749,0.0,0.229609,0.202835,0.204622,0.204639,0.221584,0.203347,0.000206,0.225262,0.20372,0.000208,0.229553
2,10505531439,0.137936,0.251644,0.200538,0.315057,0.210328,0.056868,0.072705,0.213112,0.062082,0.093859,0.091364,0.399449,0.180556,0.478871,0.210394,0.008104,0.046636,0.224054,1.416328e-09,0.036748,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.202976,0.205429,0.204756,0.221636,0.203624,0.000000,0.000000,0.203749,0.0,0.000000,0.202835,0.204622,0.204639,0.221584,0.203347,0.000206,0.000207,0.20372,0.000208,0.000207
3,12309121439,0.166742,0.192241,0.210715,0.120472,0.231834,0.070246,0.131997,0.189260,0.018200,0.068898,0.120844,0.135498,0.212009,0.060592,0.285501,0.009258,0.084712,0.167730,2.522110e-10,0.024995,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.202976,0.205429,0.204756,0.221636,0.203624,0.257463,0.225374,0.203749,0.0,0.229609,0.202835,0.204622,0.204639,0.221584,0.203347,0.257360,0.225262,0.20372,0.000208,0.229553
4,13634651439,0.177301,0.215344,0.200381,0.168308,0.211474,0.100343,0.145285,0.191114,0.115662,0.179885,0.139033,0.227013,0.181250,0.116762,0.215963,0.014037,0.095123,0.172952,2.574307e-08,0.118829,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.202976,0.205429,0.204756,0.221636,0.203624,0.257463,0.225374,0.203749,0.0,0.229609,0.202835,0.204622,0.204639,0.221584,0.203347,0.257360,0.225262,0.20372,0.000208,0.229553
5,13634651439,0.177235,0.215426,0.200238,0.184674,0.211443,0.104338,0.145285,0.191082,0.115662,0.175266,0.138937,0.226927,0.180427,0.144297,0.215973,0.015720,0.095163,0.172479,2.574307e-08,0.114183,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.202976,0.205429,0.204756,0.221636,0.203624,0.257463,0.225374,0.203749,0.0,0.229609,0.202835,0.204622,0.204639,0.221584,0.203347,0.257360,0.225262,0.20372,0.000208,0.229553


In [198]:
final_results_rf.shape

(60213, 51)

In [199]:
final_results_rf.to_csv('rf_final_results_3.csv', index=False)

In [200]:
abc = pd.read_csv('rf_final_results_3.csv')
abc.shape

(60213, 51)

In [201]:
final_results_rf = reduce_mem_usage(final_results_rf)

Mem. usage decreased to  6.66 Mb (72.1% reduction)


## Campaign Score Calculation

In [203]:
final_results_rf.head()

,SubscriberID,prob_results_rf_1,prob_results_rf_2,prob_results_rf_3,prob_results_rf_4,prob_results_rf_5,prob_results_rf_6,prob_results_rf_7,prob_results_rf_8,prob_results_rf_9,prob_results_rf_10,prob_results_crf_1,prob_results_crf_2,prob_results_crf_3,prob_results_crf_4,prob_results_crf_5,prob_results_crf_6,prob_results_rcf_7,prob_results_crf_8,prob_results_crf_9,prob_results_crf_10,prob_results_nb_1,prob_results_nb_2,prob_results_nb_3,prob_results_nb_4,prob_results_nb_5,prob_results_nb_6,prob_results_nb_7,prob_results_nb_8,prob_results_nb_9,prob_results_nb_10,prob_results_sigmoid_nb_1,prob_results_sigmoid_nb_2,prob_results_sigmoid_nb_3,prob_results_sigmoid_nb_4,prob_results_sigmoid_nb_5,prob_results_sigmoid_nb_6,prob_results_sigmoid_nb_7,prob_results_sigmoid_nb_8,prob_results_sigmoid_nb_9,prob_results_sigmoid_nb_10,prob_results_cal_sigmoid_nb_1,prob_results_cal_sigmoid_nb_2,prob_results_cal_sigmoid_nb_3,prob_results_cal_sigmoid_nb_4,prob_results_cal_sigmoid_nb_5,prob_results_cal_sigmoid_nb_6,prob_results_cal_sigmoid_nb_7,prob_results_cal_sigmoid_nb_8,prob_results_cal_sigmoid_nb_9,prob_results_cal_sigmoid_nb_10
1,4629739439,0.260010,0.217529,0.185547,0.238281,0.202026,0.067261,0.170410,0.190430,0.036865,0.163208,0.333984,0.233032,0.136475,0.242065,0.185059,0.009460,0.121826,0.171265,0.0,0.095276,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.203003,0.205444,0.204712,0.22168,0.203613,0.000000,0.225342,0.203735,0.0,0.229614,0.202881,0.20459,0.20459,0.221558,0.203369,0.000206,0.225220,0.203735,0.000208,0.229492
2,10505531439,0.137939,0.251709,0.200562,0.314941,0.210327,0.056854,0.072693,0.213135,0.062073,0.093872,0.091370,0.399414,0.180542,0.478760,0.210449,0.008102,0.046631,0.223999,0.0,0.036743,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.203003,0.205444,0.204712,0.22168,0.203613,0.000000,0.000000,0.203735,0.0,0.000000,0.202881,0.20459,0.20459,0.221558,0.203369,0.000206,0.000207,0.203735,0.000208,0.000207
3,12309121439,0.166748,0.192261,0.210693,0.120483,0.231812,0.070251,0.131958,0.189209,0.018204,0.068909,0.120850,0.135498,0.212036,0.060577,0.285400,0.009254,0.084717,0.167725,0.0,0.024994,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.203003,0.205444,0.204712,0.22168,0.203613,0.257568,0.225342,0.203735,0.0,0.229614,0.202881,0.20459,0.20459,0.221558,0.203369,0.257324,0.225220,0.203735,0.000208,0.229492
4,13634651439,0.177246,0.215332,0.200439,0.168335,0.211426,0.100342,0.145264,0.191162,0.115662,0.179932,0.139038,0.227051,0.181274,0.116760,0.215942,0.014038,0.095093,0.172974,0.0,0.118835,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.203003,0.205444,0.204712,0.22168,0.203613,0.257568,0.225342,0.203735,0.0,0.229614,0.202881,0.20459,0.20459,0.221558,0.203369,0.257324,0.225220,0.203735,0.000208,0.229492
5,13634651439,0.177246,0.215454,0.200195,0.184692,0.211426,0.104309,0.145264,0.191040,0.115662,0.175293,0.138916,0.226929,0.180420,0.144287,0.215942,0.015717,0.095154,0.172485,0.0,0.114197,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.203003,0.205444,0.204712,0.22168,0.203613,0.257568,0.225342,0.203735,0.0,0.229614,0.202881,0.20459,0.20459,0.221558,0.203369,0.257324,0.225220,0.203735,0.000208,0.229492


In [206]:
# DataFrame columns
dataframe_cols = final_results_rf.columns

# campaign specific columns
camp1_columns = [col for col in dataframe_cols if col.endswith('_1')]
camp2_columns = [col for col in dataframe_cols if col.endswith('_2')]
camp3_columns = [col for col in dataframe_cols if col.endswith('_3')]
camp4_columns = [col for col in dataframe_cols if col.endswith('_4')]
camp5_columns = [col for col in dataframe_cols if col.endswith('_5')]
camp6_columns = [col for col in dataframe_cols if col.endswith('_6')]
camp7_columns = [col for col in dataframe_cols if col.endswith('_7')]
camp8_columns = [col for col in dataframe_cols if col.endswith('_8')]
camp9_columns = [col for col in dataframe_cols if col.endswith('_9')]
camp10_columns = [col for col in dataframe_cols if col.endswith('_10')]


# display cols
print(camp1_columns)


['prob_results_rf_1', 'prob_results_crf_1', 'prob_results_nb_1', 'prob_results_sigmoid_nb_1', 'prob_results_cal_sigmoid_nb_1']


In [217]:
# Accuracy lists
rf = 0.84
crf = 0.85
nb = 0.33
snb = 0.82
cnb = 0.82

In [221]:
# Method for scoring the predictions
def calc_cancellation_score(model1, model2, model3, model4, model5,
                            acc1, acc2, acc3, acc4, acc5):
    campaign_score = []
    campaign_score = model1 * float(acc1) + model2 * float(acc2) +  model3 * float(acc3) + model4 * float(acc4) + model5 * float(acc5)
    #campaign_score = model1 * acc1 + model2 * acc2 +  model3 * acc3 + model4 * acc4 + model5 * acc5

    def NormalizeData(data):
        return (data - np.min(data)) / (np.max(data) - np.min(data))

    campaign_score = NormalizeData(campaign_score)

    return campaign_score

In [225]:
%%time
campaign_1_score = calc_cancellation_score(final_results_rf[camp1_columns[0]], final_results_rf[camp1_columns[1]], final_results_rf[camp1_columns[2]],
                                         final_results_rf[camp1_columns[3]], final_results_rf[camp1_columns[4]],
                                         rf, crf, nb, snb, cnb)
campaign_2_score = calc_cancellation_score(final_results_rf[camp2_columns[0]], final_results_rf[camp2_columns[1]], final_results_rf[camp2_columns[2]],
                                         final_results_rf[camp2_columns[3]], final_results_rf[camp2_columns[4]],
                                         rf, crf, nb, snb, cnb)
campaign_3_score = calc_cancellation_score(final_results_rf[camp3_columns[0]], final_results_rf[camp3_columns[1]], final_results_rf[camp3_columns[2]],
                                         final_results_rf[camp3_columns[3]], final_results_rf[camp3_columns[4]],
                                         rf, crf, nb, snb, cnb)
campaign_4_score = calc_cancellation_score(final_results_rf[camp4_columns[0]], final_results_rf[camp4_columns[1]], final_results_rf[camp4_columns[2]],
                                         final_results_rf[camp4_columns[3]], final_results_rf[camp4_columns[4]],
                                         rf, crf, nb, snb, cnb)
campaign_5_score = calc_cancellation_score(final_results_rf[camp5_columns[0]], final_results_rf[camp5_columns[1]], final_results_rf[camp5_columns[2]],
                                         final_results_rf[camp5_columns[3]], final_results_rf[camp5_columns[4]],
                                         rf, crf, nb, snb, cnb)
campaign_6_score = calc_cancellation_score(final_results_rf[camp6_columns[0]], final_results_rf[camp6_columns[1]], final_results_rf[camp6_columns[2]],
                                         final_results_rf[camp6_columns[3]], final_results_rf[camp6_columns[4]],
                                         rf, crf, nb, snb, cnb)
campaign_7_score = calc_cancellation_score(final_results_rf[camp7_columns[0]], final_results_rf[camp7_columns[1]], final_results_rf[camp7_columns[2]],
                                         final_results_rf[camp7_columns[3]], final_results_rf[camp7_columns[4]],
                                         rf, crf, nb, snb, cnb)
campaign_8_score = calc_cancellation_score(final_results_rf[camp8_columns[0]], final_results_rf[camp8_columns[1]], final_results_rf[camp8_columns[2]],
                                         final_results_rf[camp8_columns[3]], final_results_rf[camp8_columns[4]],
                                         rf, crf, nb, snb, cnb)
campaign_9_score = calc_cancellation_score(final_results_rf[camp9_columns[0]], final_results_rf[camp9_columns[1]], final_results_rf[camp9_columns[2]],
                                         final_results_rf[camp9_columns[3]], final_results_rf[camp9_columns[4]],
                                         rf, crf, nb, snb, cnb)
campaign_10_score = calc_cancellation_score(final_results_rf[camp10_columns[0]], final_results_rf[camp10_columns[1]], final_results_rf[camp10_columns[2]],
                                         final_results_rf[camp10_columns[3]], final_results_rf[camp10_columns[4]],
                                         rf, crf, nb, snb, cnb)

CPU times: user 229 ms, sys: 0 ns, total: 229 ms
Wall time: 237 ms


In [226]:
final_results_rf['campaign_1_score'] = campaign_1_score
final_results_rf['campaign_2_score'] = campaign_2_score
final_results_rf['campaign_3_score'] = campaign_3_score
final_results_rf['campaign_4_score'] = campaign_4_score
final_results_rf['campaign_5_score'] = campaign_5_score
final_results_rf['campaign_6_score'] = campaign_6_score
final_results_rf['campaign_7_score'] = campaign_7_score
final_results_rf['campaign_8_score'] = campaign_8_score
final_results_rf['campaign_9_score'] = campaign_9_score
final_results_rf['campaign_10_score'] = campaign_10_score
final_results_rf.head()

,SubscriberID,prob_results_rf_1,prob_results_rf_2,prob_results_rf_3,prob_results_rf_4,prob_results_rf_5,prob_results_rf_6,prob_results_rf_7,prob_results_rf_8,prob_results_rf_9,prob_results_rf_10,prob_results_crf_1,prob_results_crf_2,prob_results_crf_3,prob_results_crf_4,prob_results_crf_5,prob_results_crf_6,prob_results_rcf_7,prob_results_crf_8,prob_results_crf_9,prob_results_crf_10,prob_results_nb_1,prob_results_nb_2,prob_results_nb_3,prob_results_nb_4,prob_results_nb_5,prob_results_nb_6,prob_results_nb_7,prob_results_nb_8,prob_results_nb_9,prob_results_nb_10,prob_results_sigmoid_nb_1,prob_results_sigmoid_nb_2,prob_results_sigmoid_nb_3,prob_results_sigmoid_nb_4,prob_results_sigmoid_nb_5,prob_results_sigmoid_nb_6,prob_results_sigmoid_nb_7,prob_results_sigmoid_nb_8,prob_results_sigmoid_nb_9,prob_results_sigmoid_nb_10,prob_results_cal_sigmoid_nb_1,prob_results_cal_sigmoid_nb_2,prob_results_cal_sigmoid_nb_3,prob_results_cal_sigmoid_nb_4,prob_results_cal_sigmoid_nb_5,prob_results_cal_sigmoid_nb_6,prob_results_cal_sigmoid_nb_7,prob_results_cal_sigmoid_nb_8,prob_results_cal_sigmoid_nb_9,prob_results_cal_sigmoid_nb_10,campaign_1_score,campaign_2_score,campaign_3_score,campaign_4_score,campaign_5_score,campaign_6_score,campaign_7_score,campaign_8_score,campaign_9_score,campaign_10_score
1,4629739439,0.260010,0.217529,0.185547,0.238281,0.202026,0.067261,0.170410,0.190430,0.036865,0.163208,0.333984,0.233032,0.136475,0.242065,0.185059,0.009460,0.121826,0.171265,0.0,0.095276,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.203003,0.205444,0.204712,0.22168,0.203613,0.000000,0.225342,0.203735,0.0,0.229614,0.202881,0.20459,0.20459,0.221558,0.203369,0.000206,0.225220,0.203735,0.000208,0.229492,0.525879,0.543457,0.472656,0.527344,0.456543,0.025803,0.419922,0.459961,0.008980,0.432617
2,10505531439,0.137939,0.251709,0.200562,0.314941,0.210327,0.056854,0.072693,0.213135,0.062073,0.093872,0.091370,0.399414,0.180542,0.478760,0.210449,0.008102,0.046631,0.223999,0.0,0.036743,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.203003,0.205444,0.204712,0.22168,0.203613,0.000000,0.000000,0.203735,0.0,0.000000,0.202881,0.20459,0.20459,0.221558,0.203369,0.000206,0.000207,0.203735,0.000208,0.000207,0.366699,0.643066,0.500977,0.658691,0.471680,0.021301,0.026840,0.493408,0.015656,0.037109
3,12309121439,0.166748,0.192261,0.210693,0.120483,0.231812,0.070251,0.131958,0.189209,0.018204,0.068909,0.120850,0.135498,0.212036,0.060577,0.285400,0.009254,0.084717,0.167725,0.0,0.024994,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.203003,0.205444,0.204712,0.22168,0.203613,0.257568,0.225342,0.203735,0.0,0.229614,0.202881,0.20459,0.20459,0.221558,0.203369,0.257324,0.225220,0.203735,0.000208,0.229492,0.392090,0.481934,0.521973,0.402588,0.515137,0.370850,0.390381,0.457764,0.004044,0.365234
4,13634651439,0.177246,0.215332,0.200439,0.168335,0.211426,0.100342,0.145264,0.191162,0.115662,0.179932,0.139038,0.227051,0.181274,0.116760,0.215942,0.014038,0.095093,0.172974,0.0,0.118835,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.203003,0.205444,0.204712,0.22168,0.203613,0.257568,0.225342,0.203735,0.0,0.229614,0.202881,0.20459,0.20459,0.221558,0.203369,0.257324,0.225220,0.203735,0.000208,0.229492,0.404785,0.539551,0.501465,0.446045,0.474609,0.384033,0.399902,0.460938,0.029846,0.449219
5,13634651439,0.177246,0.215454,0.200195,0.184692,0.211426,0.104309,0.145264,0.191040,0.115662,0.175293,0.138916,0.226929,0.180420,0.144287,0.215942,0.015717,0.095154,0.172485,0.0,0.114197,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.203003,0.205444,0.204712,0.22168,0.203613,0.257568,0.225342,0.203735,0.0,0.229614,0.202881,0.20459,0.20459,0.221558,0.203369,0.257324,0.225220,0.203735,0.000208,0.229492,0.404541,0.539551,0.500977,0.464355,0.474609,0.386475,0.399902,0.460693,0.029846,0.445557


In [240]:
final_results_rf.columns

Index(['SubscriberID', 'prob_results_rf_1', 'prob_results_rf_2',
       'prob_results_rf_3', 'prob_results_rf_4', 'prob_results_rf_5',
       'prob_results_rf_6', 'prob_results_rf_7', 'prob_results_rf_8',
       'prob_results_rf_9', 'prob_results_rf_10', 'prob_results_crf_1',
       'prob_results_crf_2', 'prob_results_crf_3', 'prob_results_crf_4',
       'prob_results_crf_5', 'prob_results_crf_6', 'prob_results_rcf_7',
       'prob_results_crf_8', 'prob_results_crf_9', 'prob_results_crf_10',
       'prob_results_nb_1', 'prob_results_nb_2', 'prob_results_nb_3',
       'prob_results_nb_4', 'prob_results_nb_5', 'prob_results_nb_6',
       'prob_results_nb_7', 'prob_results_nb_8', 'prob_results_nb_9',
       'prob_results_nb_10', 'prob_results_sigmoid_nb_1',
       'prob_results_sigmoid_nb_2', 'prob_results_sigmoid_nb_3',
       'prob_results_sigmoid_nb_4', 'prob_results_sigmoid_nb_5',
       'prob_results_sigmoid_nb_6', 'prob_results_sigmoid_nb_7',
       'prob_results_sigmoid_nb_8', 'pro

In [227]:
final_results_rf.to_csv('rf_final_results_4.csv', index=False)

In [16]:
final_results_rf = pd.read_csv('rf_final_results_4.csv')
final_results_rf.head()

,SubscriberID,prob_results_rf_1,prob_results_rf_2,prob_results_rf_3,prob_results_rf_4,prob_results_rf_5,prob_results_rf_6,prob_results_rf_7,prob_results_rf_8,prob_results_rf_9,prob_results_rf_10,prob_results_crf_1,prob_results_crf_2,prob_results_crf_3,prob_results_crf_4,prob_results_crf_5,prob_results_crf_6,prob_results_rcf_7,prob_results_crf_8,prob_results_crf_9,prob_results_crf_10,prob_results_nb_1,prob_results_nb_2,prob_results_nb_3,prob_results_nb_4,prob_results_nb_5,prob_results_nb_6,prob_results_nb_7,prob_results_nb_8,prob_results_nb_9,prob_results_nb_10,prob_results_sigmoid_nb_1,prob_results_sigmoid_nb_2,prob_results_sigmoid_nb_3,prob_results_sigmoid_nb_4,prob_results_sigmoid_nb_5,prob_results_sigmoid_nb_6,prob_results_sigmoid_nb_7,prob_results_sigmoid_nb_8,prob_results_sigmoid_nb_9,prob_results_sigmoid_nb_10,prob_results_cal_sigmoid_nb_1,prob_results_cal_sigmoid_nb_2,prob_results_cal_sigmoid_nb_3,prob_results_cal_sigmoid_nb_4,prob_results_cal_sigmoid_nb_5,prob_results_cal_sigmoid_nb_6,prob_results_cal_sigmoid_nb_7,prob_results_cal_sigmoid_nb_8,prob_results_cal_sigmoid_nb_9,prob_results_cal_sigmoid_nb_10,campaign_1_score,campaign_2_score,campaign_3_score,campaign_4_score,campaign_5_score,campaign_6_score,campaign_7_score,campaign_8_score,campaign_9_score,campaign_10_score
0,4629739439,0.2600,0.2175,0.1855,0.2383,0.2020,0.06726,0.1704,0.1904,0.03687,0.1632,0.33400,0.2330,0.1365,0.24210,0.1850,0.009460,0.12180,0.1713,0.0,0.09530,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.203,0.2054,0.2047,0.2217,0.2036,0.0000,0.2253,0.2037,0.0,0.2296,0.2029,0.2046,0.2046,0.2216,0.2034,0.000206,0.225200,0.2037,0.000208,0.229500,0.5260,0.5435,0.4727,0.5273,0.4565,0.0258,0.42000,0.4600,0.008980,0.4326
1,10505531439,0.1380,0.2517,0.2006,0.3150,0.2103,0.05685,0.0727,0.2131,0.06207,0.0939,0.09140,0.3994,0.1805,0.47880,0.2104,0.008100,0.04663,0.2240,0.0,0.03674,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.203,0.2054,0.2047,0.2217,0.2036,0.0000,0.0000,0.2037,0.0,0.0000,0.2029,0.2046,0.2046,0.2216,0.2034,0.000206,0.000207,0.2037,0.000208,0.000207,0.3667,0.6430,0.5010,0.6587,0.4717,0.0213,0.02684,0.4934,0.015660,0.0371
2,12309121439,0.1667,0.1923,0.2107,0.1205,0.2318,0.07025,0.1320,0.1892,0.01820,0.0689,0.12085,0.1355,0.2120,0.06058,0.2854,0.009254,0.08470,0.1677,0.0,0.02500,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.203,0.2054,0.2047,0.2217,0.2036,0.2576,0.2253,0.2037,0.0,0.2296,0.2029,0.2046,0.2046,0.2216,0.2034,0.257300,0.225200,0.2037,0.000208,0.229500,0.3920,0.4820,0.5220,0.4026,0.5150,0.3708,0.39040,0.4578,0.004044,0.3652
3,13634651439,0.1772,0.2153,0.2004,0.1683,0.2114,0.10034,0.1453,0.1912,0.11566,0.1799,0.13900,0.2270,0.1813,0.11676,0.2160,0.014040,0.09510,0.1730,0.0,0.11884,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.203,0.2054,0.2047,0.2217,0.2036,0.2576,0.2253,0.2037,0.0,0.2296,0.2029,0.2046,0.2046,0.2216,0.2034,0.257300,0.225200,0.2037,0.000208,0.229500,0.4048,0.5396,0.5015,0.4460,0.4746,0.3840,0.40000,0.4610,0.029850,0.4492
4,13634651439,0.1772,0.2155,0.2002,0.1847,0.2114,0.10430,0.1453,0.1910,0.11566,0.1753,0.13890,0.2269,0.1804,0.14430,0.2160,0.015720,0.09515,0.1725,0.0,0.11420,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.203,0.2054,0.2047,0.2217,0.2036,0.2576,0.2253,0.2037,0.0,0.2296,0.2029,0.2046,0.2046,0.2216,0.2034,0.257300,0.225200,0.2037,0.000208,0.229500,0.4045,0.5396,0.5010,0.4644,0.4746,0.3865,0.40000,0.4607,0.029850,0.4456


In [17]:
final_results_rf = reduce_mem_usage(final_results_rf)

Mem. usage decreased to  7.35 Mb (73.8% reduction)


In [18]:
final_results_rf = final_results_rf[['SubscriberID', 'campaign_1_score', 'campaign_2_score',
                                     'campaign_3_score', 'campaign_4_score', 'campaign_5_score',
                                     'campaign_6_score', 'campaign_7_score', 'campaign_8_score',
                                     'campaign_9_score', 'campaign_10_score']].copy()
final_results_rf.shape

(60213, 11)

In [19]:
# Campaign sütunlarını düzenle
campaign_columns = ['campaign_1_score', 'campaign_2_score', 'campaign_3_score', 'campaign_4_score',
                    'campaign_5_score', 'campaign_6_score', 'campaign_7_score', 'campaign_8_score',
                    'campaign_9_score', 'campaign_10_score']

for column in campaign_columns:
    # Her sütunu kendi içinde sırala
    sorted_indices = final_results_rf[column].sort_values(ascending=False).index
    # Sıralı indeksleri kullanarak 'SubscriberID' sütununu doldur
    final_results_rf[column] = final_results_rf['SubscriberID'].iloc[sorted_indices].values

In [20]:
final_results_rf.shape

(60213, 11)

In [25]:
final_results_rf.head(10)

,SubscriberID,campaign_1_score,campaign_2_score,campaign_3_score,campaign_4_score,campaign_5_score,campaign_6_score,campaign_7_score,campaign_8_score,campaign_9_score,campaign_10_score
0,4629739439,15863696567439,12042254421439,21183535209439,16776160997439,12995306145439,13347866703439,16353751527439,15863696567439,17933118349439,15509045583439
1,10505531439,15863696567439,17662797149439,21182831157439,7391204335439,12907160573439,25154849165439,15788302159439,15863696567439,20711746487439,15930438089439
2,12309121439,907489509439,15499636493439,15432877587439,21358313945439,17661002251439,25318549947439,12813226129439,221464717439,20848919285439,19553381031439
3,13634651439,769225865439,15499636493439,18352198783439,2744465481439,15583792437439,3268180211439,12988691533439,12941785155439,18859629051439,13067662699439
4,13634651439,16948827577439,14688859771439,13477538305439,7682468909439,17991663315439,25260287471439,13113660763439,17804611475439,6534659887439,5225649033439
5,16646429439,13365033403439,18216994723439,20982515325439,17846502569439,15684024235439,25379050613439,12761452231439,18605157713439,22038493367439,15762256581439
6,16972379439,1070946915439,18216994723439,20982515325439,17846502569439,15621241919439,15121969093439,2431153649439,14953105263439,16458637891439,21176894521439
7,18897657439,16828912745439,21178863259439,18388874677439,17112163295439,15459640255439,3645417357439,2665311783439,16208968883439,12953480241439,18698018695439
8,19884199439,18384593867439,17970150615439,17983636253439,12988561153439,16756582267439,12549258781439,16452818597439,16208968883439,6785345859439,16750454407439
9,22852517439,16846222863439,21178863259439,22051314067439,902826251439,13036658335439,581604699439,16461132495439,15462208741439,14592634985439,16067780381439


In [22]:
# 'campaign' ile başlayan kolonları döngü ile gez
for column in final_results_rf.columns:
    if column.startswith('campaign'):
        # İşlemlerinizi burada gerçekleştirin
        globals()[f"{column}_list"] = final_results_rf[column].unique()[:30].tolist()


In [24]:
print(len(campaign_1_score_list))

30


In [27]:
print(campaign_1_score_list)
#print(campaign_2_score_list)
#print(campaign_3_score_list)
#print(campaign_4_score_list)
#print(campaign_5_score_list)
#print(campaign_6_score_list)
#print(campaign_7_score_list)
#print(campaign_8_score_list)
#print(campaign_9_score_list)
#print(campaign_10_score_list)


[15863696567439, 907489509439, 769225865439, 16948827577439, 13365033403439, 1070946915439, 16828912745439, 18384593867439, 16846222863439, 3763711131439, 18808902539439, 4107275469439, 14323982649439, 18086931981439, 16028918449439, 288962443439, 2968840769439, 2049913837439, 15785225191439, 17928515935439, 16828552027439, 16162384109439, 13041091255439, 18851345575439, 17926647155439, 17988629807439, 15429192179439, 17667234415439, 14614125955439, 19125765053439]


In [28]:
most_frequent_users = pd.read_csv('sample.csv', index_col='Unnamed: 0')
most_frequent_users.shape

<ipython-input-28-641c67f464be>:1: DtypeWarning: Columns (109) have mixed types. Specify dtype option on import or set low_memory=False.
  most_frequent_users = pd.read_csv('sample.csv', index_col='Unnamed: 0')


(60213, 202)

In [29]:
most_frequent_users = most_frequent_users[['SubscriberID', 'campaign_1', 'campaign_2', 'campaign_3', 'campaign_4',
                                           'campaign_5', 'campaign_6', 'campaign_7', 'campaign_8', 'campaign_9',
                                           'campaign_10', 'campaign_11', 'campaign_12', 'campaign_13',
                                           'campaign_14', 'campaign_15', 'campaign_16']].copy()

most_frequent_users.shape


(60213, 17)

In [30]:
selected_columns = most_frequent_users[['SubscriberID', 'campaign_1', 'campaign_2', 'campaign_3', 'campaign_4',
                                         'campaign_5', 'campaign_6', 'campaign_7', 'campaign_8', 'campaign_9',
                                         'campaign_10', 'campaign_11', 'campaign_12', 'campaign_13',
                                         'campaign_14', 'campaign_15', 'campaign_16']]


# 'score' adında yeni bir sütun ekleyerek, her satırın 'SubscriberID' hariç sütunlarının toplamını hesapla
most_frequent_users['common_score'] = selected_columns.iloc[:, 1:].sum(axis=1)
most_frequent_users[most_frequent_users.campaign_1 != 0].head(10)


,SubscriberID,campaign_1,campaign_2,campaign_3,campaign_4,campaign_5,campaign_6,campaign_7,campaign_8,campaign_9,campaign_10,campaign_11,campaign_12,campaign_13,campaign_14,campaign_15,campaign_16,common_score
13,39302127439,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
26,117456245439,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
39,156718009439,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
57,221464717439,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2
85,344317445439,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
121,445214181439,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
220,745679237439,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
250,858484013439,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
280,934560743439,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
312,1070946915439,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2


In [31]:
# Sorting acording to the 'common_score'
sorted_df = most_frequent_users.sort_values(by='common_score', ascending=False)

# Belirtilen kampanya sütunlarını sırasıyla 'SubscriberID' değerleriyle doldur
for col in ['campaign_11', 'campaign_12', 'campaign_13', 'campaign_14', 'campaign_15', 'campaign_16']:
    sorted_df[col] = sorted_df['SubscriberID']
sorted_df = sorted_df.drop(columns=['campaign_1', 'campaign_2', 'campaign_3', 'campaign_4',
                        'campaign_5', 'campaign_6', 'campaign_7', 'campaign_8', 'campaign_9','campaign_10'])
sorted_df.head()

,SubscriberID,campaign_11,campaign_12,campaign_13,campaign_14,campaign_15,campaign_16,common_score
36804,17840922305439,17840922305439,17840922305439,17840922305439,17840922305439,17840922305439,17840922305439,2
2261,6223825275439,6223825275439,6223825275439,6223825275439,6223825275439,6223825275439,6223825275439,2
50397,18979869833439,18979869833439,18979869833439,18979869833439,18979869833439,18979869833439,18979869833439,2
59372,22035990071439,22035990071439,22035990071439,22035990071439,22035990071439,22035990071439,22035990071439,2
665,3362644867439,3362644867439,3362644867439,3362644867439,3362644867439,3362644867439,3362644867439,2


In [32]:
# 'campaign' ile başlayan kolonları döngü ile gez
for column in sorted_df.columns:
    if column.startswith('campaign'):
        # İşlemlerinizi burada gerçekleştirin
        globals()[f"{column}_list"] = sorted_df[column].unique()[:30].tolist()

In [34]:
print(len(campaign_11_list))


30


In [35]:
print(campaign_11_list)
#print(campaign_12_list)
#print(campaign_13_list)
#print(campaign_14_list)
#print(campaign_15_list)
#print(campaign_16_list)

[17840922305439, 6223825275439, 18979869833439, 22035990071439, 3362644867439, 19756417459439, 14226832165439, 18097101621439, 18266391359439, 21982894989439, 17842582477439, 19005737225439, 19760628733439, 7545565563439, 18605157713439, 7618108995439, 18589751143439, 14723797265439, 1070946915439, 19244180515439, 16842737371439, 17605299569439, 7930751543439, 14329788905439, 18270450523439, 15421551911439, 19585141599439, 13115620809439, 23047008743439, 16457473163439]


In [37]:
final_df = pd.DataFrame()
final_df['campaign_1_list'] = campaign_1_score_list
final_df['campaign_2_list'] = campaign_2_score_list
final_df['campaign_3_list'] = campaign_3_score_list
final_df['campaign_4_list'] = campaign_4_score_list
final_df['campaign_5_list'] = campaign_5_score_list
final_df['campaign_6_list'] = campaign_6_score_list
final_df['campaign_7_list'] = campaign_7_score_list
final_df['campaign_8_list'] = campaign_8_score_list
final_df['campaign_9_list'] = campaign_9_score_list
final_df['campaign_10_list'] = campaign_10_score_list
final_df['campaign_11_list'] = campaign_11_list
final_df['campaign_12_list'] = campaign_12_list
final_df['campaign_13_list'] = campaign_13_list
final_df['campaign_14_list'] = campaign_14_list
final_df['campaign_15_list'] = campaign_15_list
final_df['campaign_16_list'] = campaign_16_list
final_df.head(10)

,campaign_1_list,campaign_2_list,campaign_3_list,campaign_4_list,campaign_5_list,campaign_6_list,campaign_7_list,campaign_8_list,campaign_9_list,campaign_10_list,campaign_11_list,campaign_12_list,campaign_13_list,campaign_14_list,campaign_15_list,campaign_16_list
0,15863696567439,12042254421439,21183535209439,16776160997439,12995306145439,13347866703439,16353751527439,15863696567439,17933118349439,15509045583439,17840922305439,17840922305439,17840922305439,17840922305439,17840922305439,17840922305439
1,907489509439,17662797149439,21182831157439,7391204335439,12907160573439,25154849165439,15788302159439,221464717439,20711746487439,15930438089439,6223825275439,6223825275439,6223825275439,6223825275439,6223825275439,6223825275439
2,769225865439,15499636493439,15432877587439,21358313945439,17661002251439,25318549947439,12813226129439,12941785155439,20848919285439,19553381031439,18979869833439,18979869833439,18979869833439,18979869833439,18979869833439,18979869833439
3,16948827577439,14688859771439,18352198783439,2744465481439,15583792437439,3268180211439,12988691533439,17804611475439,18859629051439,13067662699439,22035990071439,22035990071439,22035990071439,22035990071439,22035990071439,22035990071439
4,13365033403439,18216994723439,13477538305439,7682468909439,17991663315439,25260287471439,13113660763439,18605157713439,6534659887439,5225649033439,3362644867439,3362644867439,3362644867439,3362644867439,3362644867439,3362644867439
5,1070946915439,21178863259439,20982515325439,17846502569439,15684024235439,25379050613439,12761452231439,14953105263439,22038493367439,15762256581439,19756417459439,19756417459439,19756417459439,19756417459439,19756417459439,19756417459439
6,16828912745439,17970150615439,18388874677439,17112163295439,15621241919439,15121969093439,2431153649439,16208968883439,16458637891439,21176894521439,14226832165439,14226832165439,14226832165439,14226832165439,14226832165439,14226832165439
7,18384593867439,7916418435439,17983636253439,12988561153439,15459640255439,3645417357439,2665311783439,15462208741439,12953480241439,18698018695439,18097101621439,18097101621439,18097101621439,18097101621439,18097101621439,18097101621439
8,16846222863439,20865416701439,22051314067439,902826251439,16756582267439,12549258781439,16452818597439,18018508557439,6785345859439,16750454407439,18266391359439,18266391359439,18266391359439,18266391359439,18266391359439,18266391359439
9,3763711131439,18755298975439,19552898625439,1214543101439,13036658335439,581604699439,16461132495439,12954401593439,14592634985439,16067780381439,21982894989439,21982894989439,21982894989439,21982894989439,21982894989439,21982894989439


In [38]:
final_df.shape

(30, 16)

In [39]:
final_df.to_csv('final_df.csv', index=False)

In [40]:
abc = pd.read_csv('final_df.csv')
abc.head()

,campaign_1_list,campaign_2_list,campaign_3_list,campaign_4_list,campaign_5_list,campaign_6_list,campaign_7_list,campaign_8_list,campaign_9_list,campaign_10_list,campaign_11_list,campaign_12_list,campaign_13_list,campaign_14_list,campaign_15_list,campaign_16_list
0,15863696567439,12042254421439,21183535209439,16776160997439,12995306145439,13347866703439,16353751527439,15863696567439,17933118349439,15509045583439,17840922305439,17840922305439,17840922305439,17840922305439,17840922305439,17840922305439
1,907489509439,17662797149439,21182831157439,7391204335439,12907160573439,25154849165439,15788302159439,221464717439,20711746487439,15930438089439,6223825275439,6223825275439,6223825275439,6223825275439,6223825275439,6223825275439
2,769225865439,15499636493439,15432877587439,21358313945439,17661002251439,25318549947439,12813226129439,12941785155439,20848919285439,19553381031439,18979869833439,18979869833439,18979869833439,18979869833439,18979869833439,18979869833439
3,16948827577439,14688859771439,18352198783439,2744465481439,15583792437439,3268180211439,12988691533439,17804611475439,18859629051439,13067662699439,22035990071439,22035990071439,22035990071439,22035990071439,22035990071439,22035990071439
4,13365033403439,18216994723439,13477538305439,7682468909439,17991663315439,25260287471439,13113660763439,18605157713439,6534659887439,5225649033439,3362644867439,3362644867439,3362644867439,3362644867439,3362644867439,3362644867439
